# In this notebook we will train the xgboost model

In [ ]:
drop_xgboost_4h = [
    'open', 'high', 'low',
    'high_low', 'high_close', 'low_close', 'typical_price',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3', 
    'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'momentum_alignment',
  
]

In [ ]:
drop_catboost_4h = [
    # Raw OHLC (keep close for analysis)
    'open', 'high', 'low',
    
    # ATR intermediate calculations
    'high_low', 'high_close', 'low_close',
    
    # Helper calculation
    'typical_price',
    
    # Derivable position flags
    'above_sma20', 'above_sma50', 'ema7_above_ema21',
    
    # Optional: Complex scenario features (test individually!)
    'bullish_scenario_1',
    'bullish_scenario_5', 'bearish_scenario_1'
]

DROP_COLS = [ 
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'ema7_above_ema21', 'macd_rising',
    'bollinger_upper', 'bollinger_lower', 'bullish_scenario_1',
    'bullish_scenario_5', 'bearish_scenario_1'
]

In [ ]:
# remmber to add drop columns list + make 250 iterations insted of 100 and also in the end print the top 5
#  results with their metrics precison , recall , f1 and not only the best one

# also remmber to add rop columns to the both the model trainig and paramter finding code

In [16]:
# Complete XGBoost GPU Optimizer with F0.5 Scoring (Fixed)
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, fbeta_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna
from optuna.samplers import TPESampler
import warnings
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

warnings.filterwarnings('ignore')

class XGBoostGPUOptimizer:
    def __init__(self, data_path, target_column='target', drop_columns=None, 
                 task_type='auto', test_size=0.2, random_state=42, 
                 optimization_metric='f0.5', use_gpu=True):
        self.data_path = data_path
        self.target_column = target_column
        self.drop_columns = drop_columns or []
        self.task_type = task_type
        self.test_size = test_size
        self.random_state = random_state
        self.optimization_metric = optimization_metric
        self.use_gpu = use_gpu
        self.best_params = None
        self.best_model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.label_encoders = {}
        self.scaler = None
        self._check_gpu_availability()

    def _check_gpu_availability(self):
        """Check if GPU is available for XGBoost"""
        if self.use_gpu:
            try:
                test_data = xgb.DMatrix(np.random.random((100, 10)), label=np.random.randint(0, 2, 100))
                test_params = {'tree_method': 'hist', 'device': 'cuda'}
                xgb.train(test_params, test_data, num_boost_round=1, verbose_eval=False)
                print("✅ GPU acceleration available and enabled!")
                print(f"   XGBoost version: {xgb.__version__}")
            except Exception as e:
                print(f"⚠️  GPU not available, falling back to CPU: {str(e)}")
                self.use_gpu = False
        else:
            print("🖥️  Using CPU-only mode")

    def load_and_preprocess_data(self):
        """Load and preprocess the dataset"""
        print("Loading dataset...")
        df = pd.read_csv(self.data_path)
        print(f"Dataset shape: {df.shape}")
        
        # Check if target column exists
        if self.target_column not in df.columns:
            raise ValueError(f"Target column '{self.target_column}' not found in dataset")
        
        # Separate features and target
        y = df[self.target_column].copy()
        columns_to_drop = self.drop_columns.copy()
        if self.target_column not in columns_to_drop:
            columns_to_drop.append(self.target_column)
        X = df.drop(columns=columns_to_drop, errors='ignore')
        
        print(f"Features shape after dropping columns: {X.shape}")
        
        # Handle missing values
        for col in X.columns:
            if X[col].dtype in ['object', 'category']:
                X[col] = X[col].fillna(X[col].mode()[0] if not X[col].mode().empty else 'unknown')
            else:
                X[col] = X[col].fillna(X[col].median())

        # Handle target missing values
        if y.isnull().sum() > 0:
            print(f"Removing {y.isnull().sum()} rows with missing target values")
            mask = ~y.isnull()
            X = X[mask]
            y = y[mask]

        # Auto-detect task type
        if self.task_type == 'auto':
            unique_values = len(y.unique())
            if unique_values <= 20 and y.dtype in ['int64', 'int32', 'object', 'category']:
                self.task_type = 'classification'
                print(f"Auto-detected: Classification ({unique_values} classes)")
            else:
                self.task_type = 'regression'
                print(f"Auto-detected: Regression")

        # Encode categorical variables
        categorical_columns = X.select_dtypes(include=['object', 'category']).columns
        if len(categorical_columns) > 0:
            print(f"Encoding categorical columns: {list(categorical_columns)}")
            for col in categorical_columns:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))
                self.label_encoders[col] = le

        # Encode target for classification
        if self.task_type == 'classification' and y.dtype == 'object':
            le_target = LabelEncoder()
            y = le_target.fit_transform(y)
            self.label_encoders['target'] = le_target

        # Check class distribution
        if self.task_type == 'classification':
            class_counts = pd.Series(y).value_counts().sort_index()
            print(f"Class distribution: {dict(class_counts)}")

        # Chronological split for time series data
        print("🕐 Using chronological split for time series data...")
        split_idx = int(len(X) * (1 - self.test_size))
        
        self.X_train = X.iloc[:split_idx].copy()
        self.X_test = X.iloc[split_idx:].copy()
        self.y_train = y.iloc[:split_idx].copy()
        self.y_test = y.iloc[split_idx:].copy()
        
        print(f"Training set shape: {self.X_train.shape}")
        print(f"Test set shape: {self.X_test.shape}")
        
        return self.X_train, self.X_test, self.y_train, self.y_test

    def objective(self, trial):
        """Objective function for Optuna optimization with F0.5 scoring"""
        try:
            # Calculate class imbalance ratio (robust calculation)
            neg_count = np.sum(self.y_train == 0)
            pos_count = np.sum(self.y_train == 1)
            pos_ratio = (neg_count + 1e-6) / (pos_count + 1e-6)  # Avoid divide-by-zero

            # Binary classification parameters (hardcoded for 0/1 target)
            params = {
                'objective': 'binary:logistic',
                'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
                'colsample_bynode': trial.suggest_float('colsample_bynode', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'gamma': trial.suggest_float('gamma', 0, 5),
                'max_bin': trial.suggest_int('max_bin', 256, 512),
                'tree_method': 'hist',
                'device': 'cuda' if self.use_gpu else 'cpu',
                'random_state': self.random_state,
                'verbosity': 0,
                'eval_metric': 'logloss'
            }

            # Add scale_pos_weight for imbalanced data
            if pos_ratio > 1.5:
                params['scale_pos_weight'] = trial.suggest_float('scale_pos_weight', 1.0, pos_ratio * 2)

            # Time series cross-validation
            tscv = TimeSeriesSplit(n_splits=5, gap=24)
            
            precision_scores = []
            recall_scores = []
            f1_scores = []
            fbeta_scores = []
            
            for fold, (train_idx, val_idx) in enumerate(tscv.split(self.X_train)):
                X_tr, X_val = self.X_train.iloc[train_idx], self.X_train.iloc[val_idx]
                y_tr, y_val = self.y_train.iloc[train_idx], self.y_train.iloc[val_idx]

                # Create and train model
                model = xgb.XGBClassifier(**params)
                model.fit(X_tr, y_tr, verbose=False)

                # Make predictions
                y_val_pred = model.predict(X_val)
                
                # Calculate metrics (binary classification)
                fold_precision = precision_score(y_val, y_val_pred, zero_division=0)
                fold_recall = recall_score(y_val, y_val_pred, zero_division=0)
                fold_f1 = f1_score(y_val, y_val_pred, zero_division=0)
                fold_fbeta = fbeta_score(y_val, y_val_pred, beta=0.5, zero_division=0)
                
                precision_scores.append(fold_precision)
                recall_scores.append(fold_recall)
                f1_scores.append(fold_f1)
                fbeta_scores.append(fold_fbeta)

            # Calculate averages
            avg_precision = np.mean(precision_scores)
            avg_recall = np.mean(recall_scores)
            avg_f1 = np.mean(f1_scores)
            avg_fbeta = np.mean(fbeta_scores)
            
            # Print trial results
            print(f"Trial {trial.number:3d}: Precision={avg_precision:.4f} | Recall={avg_recall:.4f} | F1={avg_f1:.4f} | F0.5={avg_fbeta:.4f}")

            # Return optimization metric
            if np.isnan(fbeta_scores).any():
                return 0.0
            return avg_fbeta

        except Exception as e:
            print(f"⚠️  Trial {trial.number} failed with error: {str(e)}")
            return 0.0

    def optimize_hyperparameters(self, n_trials=100, timeout=3600):
        """Optimize hyperparameters using Optuna"""
        print(f"Starting F0.5-optimized hyperparameter search...")
        print(f"F0.5 gives 2x weight to precision vs recall - perfect for trading!")
        print(f"Number of trials: {n_trials}")
        print(f"Timeout: {timeout} seconds")
        
        direction = 'maximize'
        study = optuna.create_study(
            direction=direction, 
            sampler=TPESampler(seed=self.random_state),
            study_name='xgboost_f05_optimization'
        )
        
        study.optimize(self.objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)
        
        self.best_params = study.best_params
        print(f"\\nOptimization completed!")
        print(f"Best F0.5 score: {study.best_value:.6f}")
        print(f"Best parameters: {self.best_params}")
        
        return study

    def train_best_model(self):
        """Train the final model with best parameters"""
        print("\\nTraining final model with best F0.5 parameters...")
        
        # Prepare final parameters (binary classification)
        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'binary:logistic',
            'tree_method': 'hist',
            'device': 'cuda' if self.use_gpu else 'cpu',
            'random_state': self.random_state,
            'verbosity': 1
        })
            
        self.best_model = xgb.XGBClassifier(**final_params)
        self.best_model.fit(self.X_train, self.y_train)
        
        return self.best_model

    def evaluate_model(self):
        """Evaluate the trained model"""
        if self.best_model is None:
            raise ValueError("Model not trained yet. Call train_best_model() first.")
        
        print("\\n" + "="*60)
        print("F0.5-OPTIMIZED MODEL EVALUATION")
        print("="*60)
        
        # Make predictions
        y_train_pred = self.best_model.predict(self.X_train)
        y_test_pred = self.best_model.predict(self.X_test)
        
        # Calculate metrics (binary classification - 0/1 only)
        train_metrics = {
            'accuracy': accuracy_score(self.y_train, y_train_pred),
            'precision': precision_score(self.y_train, y_train_pred, zero_division=0),
            'recall': recall_score(self.y_train, y_train_pred, zero_division=0),
            'f1': f1_score(self.y_train, y_train_pred, zero_division=0),
            'f0.5': fbeta_score(self.y_train, y_train_pred, beta=0.5, zero_division=0)
        }
        
        test_metrics = {
            'accuracy': accuracy_score(self.y_test, y_test_pred),
            'precision': precision_score(self.y_test, y_test_pred, zero_division=0),
            'recall': recall_score(self.y_test, y_test_pred, zero_division=0),
            'f1': f1_score(self.y_test, y_test_pred, zero_division=0),
            'f0.5': fbeta_score(self.y_test, y_test_pred, beta=0.5, zero_division=0)
        }
        
        # Print results
        print("F0.5-OPTIMIZED METRICS:")
        print("-" * 40)
        print(f"{'Metric':<12} {'Training':<12} {'Test':<12} {'Difference':<12}")
        print("-" * 48)
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'f0.5']:
            train_val = train_metrics[metric]
            test_val = test_metrics[metric]
            diff = abs(train_val - test_val)
            print(f"{metric:<12} {train_val:<12.6f} {test_val:<12.6f} {diff:<12.6f}")
        
        print("\\n🎯 KEY METRICS (Test Set):")
        print(f"   • Precision: {test_metrics['precision']:.6f}")
        print(f"   • F0.5:      {test_metrics['f0.5']:.6f} (precision-weighted)")
        print(f"   • F1:        {test_metrics['f1']:.6f} (balanced)")
        
        print("\\nClassification Report (Test Set):")
        print(classification_report(self.y_test, y_test_pred))
        
        return {'train_metrics': train_metrics, 'test_metrics': test_metrics}

    def save_model_and_results(self, save_dir='xgboost_f05_results'):
        """Save the trained model and results"""
        if self.best_model is None:
            raise ValueError("Model not trained yet. Call train_best_model() first.")
        
        os.makedirs(save_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save model
        model_path = os.path.join(save_dir, f'xgboost_f05_model_{timestamp}.joblib')
        joblib.dump(self.best_model, model_path)
        print(f"F0.5-optimized model saved to: {model_path}")
        
        # Save parameters
        params_path = os.path.join(save_dir, f'best_f05_parameters_{timestamp}.txt')
        with open(params_path, 'w') as f:
            f.write(f"Optimization Metric: F0.5 (precision-weighted)\\n")
            f.write(f"GPU Enabled: {self.use_gpu}\\n")
            f.write(f"Best Parameters:\\n")
            for key, value in self.best_params.items():
                f.write(f"{key}: {value}\\n")
        
        return model_path, params_path

# Main execution function
def main():
    """Main function to run F0.5 optimization"""
    
    DATA_PATH = r"C:\\Users\\ADMIN\\Desktop\\Coding_projects\\stock_market_prediction\\Stock-Market-Prediction\\data\\processed\\gemini_btc_with_features_4h.csv"
    TARGET_COLUMN = "target"
    DROP_COLUMNS = [
    'open', 'high', 'low',
    'high_low', 'high_close', 'low_close', 'typical_price',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3', 
    'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'momentum_alignment',
  
]
    
    optimizer = XGBoostGPUOptimizer(
        data_path=DATA_PATH,
        target_column=TARGET_COLUMN,
        drop_columns=DROP_COLUMNS,
        optimization_metric="f0.5",  # F0.5 gives 2x weight to precision
        use_gpu=True
    )
    
    try:
        # Load and preprocess data
        X_train, X_test, y_train, y_test = optimizer.load_and_preprocess_data()
        
        # Optimize hyperparameters for F0.5
        study = optimizer.optimize_hyperparameters(n_trials=100, timeout=3600)
        
        # Train best model
        best_model = optimizer.train_best_model()
        
        # Evaluate model
        results = optimizer.evaluate_model()
        
        # Save results
        model_path, params_path = optimizer.save_model_and_results()
        
        print("\\n" + "="*60)
        print("F0.5 OPTIMIZATION COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"🎯 F0.5-optimized model saved to: {model_path}")
        print(f"📋 Parameters saved to: {params_path}")
        print(f"🏆 Final Test F0.5: {results['test_metrics']['f0.5']:.6f}")
        
        return optimizer, study, results
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None

if __name__ == "__main__":
    optimizer, study, results = main() 

✅ GPU acceleration available and enabled!
   XGBoost version: 3.0.2
Loading dataset...
Dataset shape: (20718, 67)
Features shape after dropping columns: (20718, 54)
Auto-detected: Classification (2 classes)
Encoding categorical columns: ['timestamp']
Class distribution: {0: np.int64(9967), 1: np.int64(10751)}
🕐 Using chronological split for time series data...
Training set shape: (16574, 54)
Test set shape: (4144, 54)
Starting F0.5-optimized hyperparameter search...
F0.5 gives 2x weight to precision vs recall - perfect for trading!
Number of trials: 100
Timeout: 3600 seconds


[I 2025-06-12 16:38:59,227] A new study created in memory with name: xgboost_f05_optimization
Best trial: 0. Best value: 0.52196:   1%|          | 1/100 [00:58<1:36:30, 58.49s/it, 58.49/3600 seconds]

Trial   0: Precision=0.5370 | Recall=0.5181 | F1=0.5127 | F0.5=0.5220
[I 2025-06-12 16:39:57,718] Trial 0 finished with value: 0.5219603551381751 and parameters: {'n_estimators': 812, 'max_depth': 15, 'learning_rate': 0.14553179565665345, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'colsample_bynode': 0.6232334448672797, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598, 'min_child_weight': 8, 'gamma': 0.10292247147901223, 'max_bin': 505}. Best is trial 0 with value: 0.5219603551381751.


Best trial: 0. Best value: 0.52196:   2%|▏         | 2/100 [01:57<1:35:50, 58.68s/it, 117.30/3600 seconds]

Trial   1: Precision=0.5449 | Recall=0.4658 | F1=0.4890 | F0.5=0.5172
[I 2025-06-12 16:40:56,526] Trial 1 finished with value: 0.5172440089416588 and parameters: {'n_estimators': 1682, 'max_depth': 5, 'learning_rate': 0.011551009439226469, 'subsample': 0.6733618039413735, 'colsample_bytree': 0.7216968971838151, 'colsample_bylevel': 0.8099025726528951, 'colsample_bynode': 0.7727780074568463, 'reg_alpha': 4.17890272377219e-06, 'reg_lambda': 0.0032112643094417484, 'min_child_weight': 2, 'gamma': 1.4607232426760908, 'max_bin': 350}. Best is trial 0 with value: 0.5219603551381751.


...

Best trial: 63. Best value: 0.57214:  99%|█████████▉| 99/100 [31:52<00:17, 17.48s/it, 1912.95/3600 seconds]

Trial  98: Precision=0.5334 | Recall=0.8285 | F1=0.6409 | F0.5=0.5704
[I 2025-06-12 17:10:52,172] Trial 98 finished with value: 0.5703573703715623 and parameters: {'n_estimators': 153, 'max_depth': 10, 'learning_rate': 0.0050016463516870035, 'subsample': 0.7648437723083729, 'colsample_bytree': 0.6550043886733103, 'colsample_bylevel': 0.8978287244153174, 'colsample_bynode': 0.6006227499667893, 'reg_alpha': 2.5277019263753244, 'reg_lambda': 0.009024542068296396, 'min_child_weight': 5, 'gamma': 2.6021269907013367, 'max_bin': 430}. Best is trial 63 with value: 0.5721398499838035.


Best trial: 63. Best value: 0.57214: 100%|██████████| 100/100 [32:03<00:00, 19.24s/it, 1923.58/3600 seconds]

Trial  99: Precision=0.5289 | Recall=0.8594 | F1=0.6506 | F0.5=0.5711
[I 2025-06-12 17:11:02,802] Trial 99 finished with value: 0.571062277632939 and parameters: {'n_estimators': 152, 'max_depth': 10, 'learning_rate': 0.005240578329731185, 'subsample': 0.7651730607230356, 'colsample_bytree': 0.6318385953435955, 'colsample_bylevel': 0.916547265646687, 'colsample_bynode': 0.6225529059741628, 'reg_alpha': 6.128116326056999, 'reg_lambda': 0.019433498280224788, 'min_child_weight': 5, 'gamma': 2.424062851363187, 'max_bin': 410}. Best is trial 63 with value: 0.5721398499838035.
\nOptimization completed!
Best F0.5 score: 0.572140
Best parameters: {'n_estimators': 107, 'max_depth': 7, 'learning_rate': 0.007331979140441173, 'subsample': 0.6772174715021827, 'colsample_bytree': 0.8238805270933549, 'colsample_bylevel': 0.7730466620963483, 'colsample_bynode': 0.6682429822080043, 'reg_alpha': 5.409756188443174, 'reg_lambda': 0.0015524839266472987, 'min_child_weight': 6, 'gamma': 3.7995867106505936, '

\n============================================================
F0.5-OPTIMIZED MODEL EVALUATION
F0.5-OPTIMIZED METRICS:
----------------------------------------
Metric       Training     Test         Difference  
------------------------------------------------
accuracy     0.583203     0.542712     0.040490    
precision    0.570573     0.554183     0.016390    
recall       0.805865     0.549223     0.256642    
f1           0.668108     0.551692     0.116416    
f0.5         0.605958     0.553183     0.052774    
\n🎯 KEY METRICS (Test Set):
   • Precision: 0.554183
   • F0.5:      0.553183 (precision-weighted)
   • F1:        0.551692 (balanced)
\nClassification Report (Test Set):
              precision    recall  f1-score   support

           0       0.53      0.54      0.53      2021
           1       0.55      0.55      0.55      2123

    accuracy                           0.54      4144
   macro avg       0.54      0.54      0.54      4144
weighted avg       0.54      0.54   

In [5]:
import re
import pandas as pd
import ast

def parse_xgboost_output_from_text(output_text):
    """
    Parses Optuna XGBoost log output and returns a DataFrame of trials with metrics and parameters.
    """

    # Step 1: Extract trial metrics
    trial_pattern = r'Trial\s+(\d+):\s+Precision=([0-9.]+)\s+\|\s+Recall=([0-9.]+)\s+\|\s+F1=([0-9.]+)\s+\|\s+F0\.5=([0-9.]+)'
    trial_matches = re.findall(trial_pattern, output_text)

    # Step 2: Extract trial parameters
    param_pattern = r'Trial (\d+) finished with value: ([0-9.]+) and parameters: ({.*?})\.'
    param_matches = re.findall(param_pattern, output_text)

    # Step 3: Create DataFrame for metrics
    metrics_df = pd.DataFrame(trial_matches, columns=['trial', 'precision', 'recall', 'f1', 'f0.5'])
    metrics_df = metrics_df.astype({'trial': int, 'precision': float, 'recall': float, 'f1': float, 'f0.5': float})

    # Step 4: Create DataFrame for parameters
    param_df = pd.DataFrame(param_matches, columns=['trial', 'value', 'params'])
    param_df['trial'] = param_df['trial'].astype(int)
    param_df['value'] = param_df['value'].astype(float)
    param_df['params'] = param_df['params'].apply(ast.literal_eval)
    params_expanded = param_df['params'].apply(pd.Series)

    # Step 5: Merge into final DataFrame
    final_df = pd.concat([metrics_df.set_index('trial'), param_df.set_index('trial')[['value']], params_expanded], axis=1).reset_index()

    return final_df

# ─── Paste your Optuna output here ───
output_text = """
[I 2025-06-12 15:01:31,749] A new study created in memory with name: xgboost_f05_optimization
✅ GPU acceleration available and enabled!
   XGBoost version: 3.0.2
Loading dataset...
Dataset shape: (20718, 67)
Features shape after dropping columns: (20718, 65)
Auto-detected: Classification (2 classes)
Class distribution: {0: np.int64(9967), 1: np.int64(10751)}
🕐 Using chronological split for time series data...
Training set shape: (16574, 65)
Test set shape: (4144, 65)
Starting F0.5-optimized hyperparameter search...
F0.5 gives 2x weight to precision vs recall - perfect for trading!
Number of trials: 100
Timeout: 3600 seconds
Best trial: 0. Best value: 0.528217:   1%|          | 1/100 [01:10<1:57:07, 70.98s/it, 70.98/3600 seconds]Trial   0: Precision=0.5372 | Recall=0.5152 | F1=0.5193 | F0.5=0.5282
[I 2025-06-12 15:02:42,730] Trial 0 finished with value: 0.5282168764196205 and parameters: {'n_estimators': 812, 'max_depth': 15, 'learning_rate': 0.14553179565665345, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'colsample_bynode': 0.6232334448672797, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598, 'min_child_weight': 8, 'gamma': 0.10292247147901223, 'max_bin': 505}. Best is trial 0 with value: 0.5282168764196205.
Best trial: 0. Best value: 0.528217:   2%|▏         | 2/100 [02:15<1:50:04, 67.39s/it, 135.85/3600 seconds]Trial   1: Precision=0.5404 | Recall=0.4245 | F1=0.4492 | F0.5=0.4907
[I 2025-06-12 15:03:47,602] Trial 1 finished with value: 0.4906696167401158 and parameters: {'n_estimators': 1682, 'max_depth': 5, 'learning_rate': 0.011551009439226469, 'subsample': 0.6733618039413735, 'colsample_bytree': 0.7216968971838151, 'colsample_bylevel': 0.8099025726528951, 'colsample_bynode': 0.7727780074568463, 'reg_alpha': 4.17890272377219e-06, 'reg_lambda': 0.0032112643094417484, 'min_child_weight': 2, 'gamma': 1.4607232426760908, 'max_bin': 350}. Best is trial 0 with value: 0.5282168764196205.
Best trial: 2. Best value: 0.533852:   3%|▎         | 3/100 [02:34<1:12:47, 45.03s/it, 154.27/3600 seconds]Trial   2: Precision=0.5473 | Recall=0.5291 | F1=0.5250 | F0.5=0.5339
[I 2025-06-12 15:04:06,022] Trial 2 finished with value: 0.5338518386236022 and parameters: {'n_estimators': 966, 'max_depth': 13, 'learning_rate': 0.01254057843022616, 'subsample': 0.8056937753654446, 'colsample_bytree': 0.836965827544817, 'colsample_bylevel': 0.6185801650879991, 'colsample_bynode': 0.8430179407605753, 'reg_alpha': 3.425445902633376e-07, 'reg_lambda': 3.850031979199519e-08, 'min_child_weight': 10, 'gamma': 4.828160165372797, 'max_bin': 463}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   4%|▍         | 4/100 [02:47<52:07, 32.58s/it, 167.78/3600 seconds]  Trial   3: Precision=0.5393 | Recall=0.4882 | F1=0.4959 | F0.5=0.5165
[I 2025-06-12 15:04:19,526] Trial 3 finished with value: 0.5165136478899852 and parameters: {'n_estimators': 679, 'max_depth': 4, 'learning_rate': 0.11679817513130797, 'subsample': 0.7760609974958406, 'colsample_bytree': 0.6488152939379115, 'colsample_bylevel': 0.798070764044508, 'colsample_bynode': 0.6137554084460873, 'reg_alpha': 1.527156759251193, 'reg_lambda': 2.133142332373004e-06, 'min_child_weight': 7, 'gamma': 1.5585553804470549, 'max_bin': 389}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   5%|▌         | 5/100 [03:08<45:00, 28.42s/it, 188.83/3600 seconds]Trial   4: Precision=0.5345 | Recall=0.4684 | F1=0.4848 | F0.5=0.5090
[I 2025-06-12 15:04:40,577] Trial 4 finished with value: 0.5089542030513506 and parameters: {'n_estimators': 1139, 'max_depth': 5, 'learning_rate': 0.43464957555697725, 'subsample': 0.9100531293444458, 'colsample_bytree': 0.9757995766256756, 'colsample_bylevel': 0.9579309401710595, 'colsample_bynode': 0.8391599915244341, 'reg_alpha': 1.9809253750493907, 'reg_lambda': 6.257956190096665e-08, 'min_child_weight': 2, 'gamma': 0.22613644455269033, 'max_bin': 339}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   6%|▌         | 6/100 [03:19<34:55, 22.30s/it, 199.23/3600 seconds]Trial   5: Precision=0.5338 | Recall=0.5485 | F1=0.5292 | F0.5=0.5285
[I 2025-06-12 15:04:50,978] Trial 5 finished with value: 0.5284929006087904 and parameters: {'n_estimators': 838, 'max_depth': 6, 'learning_rate': 0.22721919801680068, 'subsample': 0.7427013306774357, 'colsample_bytree': 0.7123738038749523, 'colsample_bylevel': 0.8170784332632994, 'colsample_bynode': 0.6563696899899051, 'reg_alpha': 0.16587190283399655, 'reg_lambda': 4.6876566400928895e-08, 'min_child_weight': 10, 'gamma': 3.861223846483287, 'max_bin': 307}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   7%|▋         | 7/100 [03:28<27:48, 17.94s/it, 208.21/3600 seconds]Trial   6: Precision=0.5380 | Recall=0.4559 | F1=0.4752 | F0.5=0.5041
[I 2025-06-12 15:04:59,962] Trial 6 finished with value: 0.5040869819365341 and parameters: {'n_estimators': 110, 'max_depth': 13, 'learning_rate': 0.12962378302375793, 'subsample': 0.8916028672163949, 'colsample_bytree': 0.9085081386743783, 'colsample_bylevel': 0.6296178606936361, 'colsample_bynode': 0.7433862914177091, 'reg_alpha': 1.1036250149900698e-07, 'reg_lambda': 0.5860448217200517, 'min_child_weight': 7, 'gamma': 1.654490124263246, 'max_bin': 272}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   8%|▊         | 8/100 [03:35<22:28, 14.66s/it, 215.85/3600 seconds]Trial   7: Precision=0.5390 | Recall=0.4863 | F1=0.5025 | F0.5=0.5214
[I 2025-06-12 15:05:07,595] Trial 7 finished with value: 0.5214033346569088 and parameters: {'n_estimators': 691, 'max_depth': 7, 'learning_rate': 0.14394028591544616, 'subsample': 0.8550229885420852, 'colsample_bytree': 0.9548850970305306, 'colsample_bylevel': 0.7888859700647797, 'colsample_bynode': 0.6478376983753207, 'reg_alpha': 0.02625445968759339, 'reg_lambda': 0.0703178263660987, 'min_child_weight': 6, 'gamma': 3.854835899772805, 'max_bin': 382}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:   9%|▉         | 9/100 [05:02<56:21, 37.16s/it, 302.46/3600 seconds]Trial   8: Precision=0.5438 | Recall=0.4577 | F1=0.4754 | F0.5=0.5070
[I 2025-06-12 15:06:34,213] Trial 8 finished with value: 0.5070378779378909 and parameters: {'n_estimators': 1093, 'max_depth': 8, 'learning_rate': 0.005620931047896532, 'subsample': 0.6431565707973218, 'colsample_bytree': 0.6125716742746937, 'colsample_bylevel': 0.8545641645055122, 'colsample_bynode': 0.7257423924305306, 'reg_alpha': 0.0003776906385115029, 'reg_lambda': 1.4726456718740824, 'min_child_weight': 3, 'gamma': 2.0519146151781484, 'max_bin': 450}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 2. Best value: 0.533852:  10%|█         | 10/100 [05:12<43:21, 28.91s/it, 312.91/3600 seconds]Trial   9: Precision=0.5524 | Recall=0.4606 | F1=0.4804 | F0.5=0.5125
[I 2025-06-12 15:06:44,655] Trial 9 finished with value: 0.5125455497849721 and parameters: {'n_estimators': 534, 'max_depth': 4, 'learning_rate': 0.01898772399339208, 'subsample': 0.6644885149016018, 'colsample_bytree': 0.9718790609370292, 'colsample_bylevel': 0.9232481518257668, 'colsample_bynode': 0.8533615026041694, 'reg_alpha': 0.6968596391373533, 'reg_lambda': 0.1710207048797339, 'min_child_weight': 2, 'gamma': 4.462794992449889, 'max_bin': 394}. Best is trial 2 with value: 0.5338518386236022.
Best trial: 10. Best value: 0.547337:  11%|█         | 11/100 [05:28<36:47, 24.81s/it, 328.42/3600 seconds]Trial  10: Precision=0.5403 | Recall=0.6024 | F1=0.5630 | F0.5=0.5473
[I 2025-06-12 15:07:00,167] Trial 10 finished with value: 0.5473365544016147 and parameters: {'n_estimators': 1577, 'max_depth': 11, 'learning_rate': 0.03621799474202486, 'subsample': 0.9729161367647149, 'colsample_bytree': 0.8466707575873181, 'colsample_bylevel': 0.7151465108916805, 'colsample_bynode': 0.9768664324728544, 'reg_alpha': 1.2784194237583953e-08, 'reg_lambda': 1.030757695829262e-05, 'min_child_weight': 10, 'gamma': 4.96202058341917, 'max_bin': 499}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  12%|█▏        | 12/100 [05:43<32:06, 21.89s/it, 343.63/3600 seconds]Trial  11: Precision=0.5437 | Recall=0.5924 | F1=0.5545 | F0.5=0.5446
[I 2025-06-12 15:07:15,376] Trial 11 finished with value: 0.544581718896521 and parameters: {'n_estimators': 1617, 'max_depth': 11, 'learning_rate': 0.044095880367849595, 'subsample': 0.9782637392221827, 'colsample_bytree': 0.8410499373345498, 'colsample_bylevel': 0.6871414335345349, 'colsample_bynode': 0.9801940060533646, 'reg_alpha': 1.0769715790317198e-08, 'reg_lambda': 9.546702333605166e-06, 'min_child_weight': 10, 'gamma': 4.985825626172574, 'max_bin': 500}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  13%|█▎        | 13/100 [06:03<30:39, 21.14s/it, 363.04/3600 seconds]Trial  12: Precision=0.5493 | Recall=0.5049 | F1=0.5101 | F0.5=0.5274
[I 2025-06-12 15:07:34,790] Trial 12 finished with value: 0.5274343020362775 and parameters: {'n_estimators': 1909, 'max_depth': 10, 'learning_rate': 0.04465130455121415, 'subsample': 0.9984066126989742, 'colsample_bytree': 0.8288055772301383, 'colsample_bylevel': 0.708980611666942, 'colsample_bynode': 0.9997449754566204, 'reg_alpha': 1.0509212025163454e-08, 'reg_lambda': 9.658641762306647e-06, 'min_child_weight': 9, 'gamma': 3.210999781834424, 'max_bin': 512}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  14%|█▍        | 14/100 [06:17<27:18, 19.06s/it, 377.28/3600 seconds]Trial  13: Precision=0.5444 | Recall=0.5470 | F1=0.5362 | F0.5=0.5389
[I 2025-06-12 15:07:49,032] Trial 13 finished with value: 0.53892663365751 and parameters: {'n_estimators': 1479, 'max_depth': 10, 'learning_rate': 0.04288073747566629, 'subsample': 0.9984067957444853, 'colsample_bytree': 0.8688373577594762, 'colsample_bylevel': 0.7157541529215599, 'colsample_bynode': 0.992025551184718, 'reg_alpha': 2.0673298327150042e-05, 'reg_lambda': 2.7658555339141567e-05, 'min_child_weight': 4, 'gamma': 4.9863647064750936, 'max_bin': 460}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  15%|█▌        | 15/100 [06:37<27:17, 19.26s/it, 397.02/3600 seconds]Trial  14: Precision=0.5485 | Recall=0.4860 | F1=0.4993 | F0.5=0.5230
[I 2025-06-12 15:08:08,773] Trial 14 finished with value: 0.5230459319406981 and parameters: {'n_estimators': 1448, 'max_depth': 12, 'learning_rate': 0.027967202614255386, 'subsample': 0.9383919204108254, 'colsample_bytree': 0.758730254741415, 'colsample_bylevel': 0.7253702586686225, 'colsample_bynode': 0.9287300763833096, 'reg_alpha': 1.1995224485250402e-08, 'reg_lambda': 0.00013710128828236596, 'min_child_weight': 9, 'gamma': 3.1466433301490775, 'max_bin': 432}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  16%|█▌        | 16/100 [06:52<25:22, 18.13s/it, 412.52/3600 seconds]Trial  15: Precision=0.5428 | Recall=0.5631 | F1=0.5422 | F0.5=0.5399
[I 2025-06-12 15:08:24,264] Trial 15 finished with value: 0.5398786058986382 and parameters: {'n_estimators': 1981, 'max_depth': 11, 'learning_rate': 0.07359536390791321, 'subsample': 0.9438869784567316, 'colsample_bytree': 0.7753801906386225, 'colsample_bylevel': 0.6791639002034141, 'colsample_bynode': 0.9223912814117319, 'reg_alpha': 0.0007136002774640431, 'reg_lambda': 5.560213556234732e-07, 'min_child_weight': 10, 'gamma': 4.458116274002602, 'max_bin': 485}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  17%|█▋        | 17/100 [07:08<24:11, 17.49s/it, 428.51/3600 seconds]Trial  16: Precision=0.5393 | Recall=0.5149 | F1=0.5156 | F0.5=0.5263
[I 2025-06-12 15:08:40,262] Trial 16 finished with value: 0.526272376085853 and parameters: {'n_estimators': 1341, 'max_depth': 15, 'learning_rate': 0.06742959991035824, 'subsample': 0.9610628594861361, 'colsample_bytree': 0.8966022336767454, 'colsample_bylevel': 0.7481435303576582, 'colsample_bynode': 0.9323859666124772, 'reg_alpha': 1.0160476146493027e-06, 'reg_lambda': 0.0003337139522566804, 'min_child_weight': 5, 'gamma': 3.8971410115583933, 'max_bin': 428}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  18%|█▊        | 18/100 [07:37<28:43, 21.02s/it, 457.75/3600 seconds]Trial  17: Precision=0.5452 | Recall=0.4523 | F1=0.4763 | F0.5=0.5096
[I 2025-06-12 15:09:09,499] Trial 17 finished with value: 0.5095628261005162 and parameters: {'n_estimators': 1731, 'max_depth': 9, 'learning_rate': 0.025483359760344625, 'subsample': 0.8879073586015884, 'colsample_bytree': 0.8127450154361803, 'colsample_bylevel': 0.8842800445446883, 'colsample_bynode': 0.8873114092754438, 'reg_alpha': 3.017603616190815e-05, 'reg_lambda': 5.914748802363122e-07, 'min_child_weight': 8, 'gamma': 2.857576680313304, 'max_bin': 488}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  19%|█▉        | 19/100 [08:41<45:40, 33.84s/it, 521.46/3600 seconds]Trial  18: Precision=0.5448 | Recall=0.4863 | F1=0.4904 | F0.5=0.5124
[I 2025-06-12 15:10:13,206] Trial 18 finished with value: 0.5124194924746497 and parameters: {'n_estimators': 1697, 'max_depth': 13, 'learning_rate': 0.007592065915538492, 'subsample': 0.6002600977282807, 'colsample_bytree': 0.9134822074638909, 'colsample_bylevel': 0.7557750626859493, 'colsample_bynode': 0.9496606563368475, 'reg_alpha': 6.707971818071522e-08, 'reg_lambda': 2.3621183256029898e-05, 'min_child_weight': 8, 'gamma': 4.341541203152221, 'max_bin': 423}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  20%|██        | 20/100 [10:03<1:04:26, 48.33s/it, 603.57/3600 seconds]Trial  19: Precision=0.5442 | Recall=0.4628 | F1=0.4805 | F0.5=0.5097
[I 2025-06-12 15:11:35,320] Trial 19 finished with value: 0.5096837499473906 and parameters: {'n_estimators': 1272, 'max_depth': 11, 'learning_rate': 0.030611261293365045, 'subsample': 0.8500867677135329, 'colsample_bytree': 0.7751560679816923, 'colsample_bylevel': 0.6668651828914425, 'colsample_bynode': 0.9720177690526055, 'reg_alpha': 0.003904933417253941, 'reg_lambda': 0.002080158956496785, 'min_child_weight': 9, 'gamma': 0.8217182626308466, 'max_bin': 482}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  21%|██        | 21/100 [10:38<58:18, 44.28s/it, 638.41/3600 seconds]  Trial  20: Precision=0.5482 | Recall=0.4709 | F1=0.4885 | F0.5=0.5169
[I 2025-06-12 15:12:10,158] Trial 20 finished with value: 0.5169018493210212 and parameters: {'n_estimators': 1596, 'max_depth': 8, 'learning_rate': 0.015820572538601353, 'subsample': 0.734793733388615, 'colsample_bytree': 0.8528438851069985, 'colsample_bylevel': 0.642926325310971, 'colsample_bynode': 0.8877775341309831, 'reg_alpha': 5.782407074939178e-06, 'reg_lambda': 2.090457364793104e-06, 'min_child_weight': 6, 'gamma': 3.5719975457308593, 'max_bin': 405}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  22%|██▏       | 22/100 [10:54<46:36, 35.85s/it, 654.61/3600 seconds]Trial  21: Precision=0.5386 | Recall=0.5667 | F1=0.5477 | F0.5=0.5411
[I 2025-06-12 15:12:26,360] Trial 21 finished with value: 0.5411088709011148 and parameters: {'n_estimators': 1969, 'max_depth': 11, 'learning_rate': 0.0786670815913057, 'subsample': 0.9519969561033745, 'colsample_bytree': 0.78886288321171, 'colsample_bylevel': 0.6847908321350772, 'colsample_bynode': 0.906059436409382, 'reg_alpha': 0.000453074267228923, 'reg_lambda': 3.30815919363706e-07, 'min_child_weight': 10, 'gamma': 4.462139633305128, 'max_bin': 483}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  23%|██▎       | 23/100 [11:06<36:48, 28.68s/it, 666.56/3600 seconds]Trial  22: Precision=0.5498 | Recall=0.5588 | F1=0.5403 | F0.5=0.5421
[I 2025-06-12 15:12:38,309] Trial 22 finished with value: 0.5421470924901236 and parameters: {'n_estimators': 1831, 'max_depth': 11, 'learning_rate': 0.07507866094735219, 'subsample': 0.9696966367938468, 'colsample_bytree': 0.7989907359530354, 'colsample_bylevel': 0.6853431737284235, 'colsample_bynode': 0.9611133759993724, 'reg_alpha': 7.245888910794863e-05, 'reg_lambda': 3.9301089101070274e-07, 'min_child_weight': 10, 'gamma': 4.868860945061802, 'max_bin': 509}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  24%|██▍       | 24/100 [11:20<30:50, 24.35s/it, 680.81/3600 seconds]Trial  23: Precision=0.5416 | Recall=0.5038 | F1=0.5111 | F0.5=0.5258
[I 2025-06-12 15:12:52,554] Trial 23 finished with value: 0.5257871624629149 and parameters: {'n_estimators': 1808, 'max_depth': 12, 'learning_rate': 0.05225642525665542, 'subsample': 0.9713172883525554, 'colsample_bytree': 0.7293912040754702, 'colsample_bylevel': 0.6074439390815723, 'colsample_bynode': 0.9670859135474954, 'reg_alpha': 6.536351332688057e-08, 'reg_lambda': 1.0969787584030292e-08, 'min_child_weight': 9, 'gamma': 4.769037323746385, 'max_bin': 511}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  25%|██▌       | 25/100 [11:38<27:59, 22.39s/it, 698.62/3600 seconds]Trial  24: Precision=0.5459 | Recall=0.5022 | F1=0.5114 | F0.5=0.5280
[I 2025-06-12 15:13:10,365] Trial 24 finished with value: 0.5280035946623445 and parameters: {'n_estimators': 1527, 'max_depth': 9, 'learning_rate': 0.03240688202945335, 'subsample': 0.9217274210810978, 'colsample_bytree': 0.873655217705636, 'colsample_bylevel': 0.7587693368692674, 'colsample_bynode': 0.9642249587628499, 'reg_alpha': 6.911766308732325e-07, 'reg_lambda': 7.51325578172344e-06, 'min_child_weight': 10, 'gamma': 4.130100765352471, 'max_bin': 459}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  26%|██▌       | 26/100 [11:53<24:42, 20.03s/it, 713.16/3600 seconds]Trial  25: Precision=0.5335 | Recall=0.5565 | F1=0.5408 | F0.5=0.5354
[I 2025-06-12 15:13:24,910] Trial 25 finished with value: 0.5354094662995321 and parameters: {'n_estimators': 1809, 'max_depth': 14, 'learning_rate': 0.2487956625088385, 'subsample': 0.9799346657644713, 'colsample_bytree': 0.8135695386643549, 'colsample_bylevel': 0.7058226573957213, 'colsample_bynode': 0.8074879828556969, 'reg_alpha': 0.00758407576318697, 'reg_lambda': 0.0005529213350159316, 'min_child_weight': 7, 'gamma': 4.949306978309823, 'max_bin': 491}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  27%|██▋       | 27/100 [12:10<23:22, 19.21s/it, 730.45/3600 seconds]Trial  26: Precision=0.5436 | Recall=0.4473 | F1=0.4722 | F0.5=0.5062
[I 2025-06-12 15:13:42,196] Trial 26 finished with value: 0.5062030473058672 and parameters: {'n_estimators': 1294, 'max_depth': 10, 'learning_rate': 0.09965994888932833, 'subsample': 0.8865939089993788, 'colsample_bytree': 0.9408593022802817, 'colsample_bylevel': 0.6448694721031036, 'colsample_bynode': 0.9986278773908163, 'reg_alpha': 4.053140892519333e-05, 'reg_lambda': 7.805636828283561e-05, 'min_child_weight': 8, 'gamma': 2.373136799007815, 'max_bin': 442}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  28%|██▊       | 28/100 [12:27<22:25, 18.69s/it, 747.91/3600 seconds]Trial  27: Precision=0.5502 | Recall=0.4737 | F1=0.4939 | F0.5=0.5210
[I 2025-06-12 15:13:59,659] Trial 27 finished with value: 0.5209546763451132 and parameters: {'n_estimators': 1604, 'max_depth': 12, 'learning_rate': 0.05668266011117912, 'subsample': 0.9350736624069932, 'colsample_bytree': 0.7436618712234054, 'colsample_bylevel': 0.7491855852401944, 'colsample_bynode': 0.8767492006301629, 'reg_alpha': 3.442806528643752e-08, 'reg_lambda': 2.0136915328327024e-07, 'min_child_weight': 9, 'gamma': 3.469343482663943, 'max_bin': 471}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  29%|██▉       | 29/100 [12:41<20:11, 17.07s/it, 761.21/3600 seconds]Trial  28: Precision=0.5461 | Recall=0.5682 | F1=0.5477 | F0.5=0.5439
[I 2025-06-12 15:14:12,953] Trial 28 finished with value: 0.543888772853445 and parameters: {'n_estimators': 1350, 'max_depth': 9, 'learning_rate': 0.03788203546967142, 'subsample': 0.9774554920586199, 'colsample_bytree': 0.8773394984414489, 'colsample_bylevel': 0.9967825861452994, 'colsample_bynode': 0.950421444463226, 'reg_alpha': 3.539659322651877e-07, 'reg_lambda': 2.442929840236602e-06, 'min_child_weight': 10, 'gamma': 4.603738622883117, 'max_bin': 498}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  30%|███       | 30/100 [13:04<21:56, 18.81s/it, 784.07/3600 seconds]Trial  29: Precision=0.5543 | Recall=0.5116 | F1=0.5195 | F0.5=0.5361
[I 2025-06-12 15:14:35,814] Trial 29 finished with value: 0.5361353641563273 and parameters: {'n_estimators': 1364, 'max_depth': 9, 'learning_rate': 0.02109450782917136, 'subsample': 0.826708231624751, 'colsample_bytree': 0.8833522446713408, 'colsample_bylevel': 0.975466640084087, 'colsample_bynode': 0.9292240818671718, 'reg_alpha': 3.11982695395809e-07, 'reg_lambda': 2.7256612087719455e-06, 'min_child_weight': 8, 'gamma': 4.1513711291412285, 'max_bin': 495}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 10. Best value: 0.547337:  31%|███       | 31/100 [13:17<19:40, 17.11s/it, 797.23/3600 seconds]Trial  30: Precision=0.5436 | Recall=0.4558 | F1=0.4769 | F0.5=0.5090
[I 2025-06-12 15:14:48,984] Trial 30 finished with value: 0.5089636579245829 and parameters: {'n_estimators': 1193, 'max_depth': 8, 'learning_rate': 0.03750337014603532, 'subsample': 0.9124325426089073, 'colsample_bytree': 0.9385310758883566, 'colsample_bylevel': 0.9983673217562125, 'colsample_bynode': 0.8127492272052791, 'reg_alpha': 1.837949035927626e-06, 'reg_lambda': 0.014097942328666799, 'min_child_weight': 1, 'gamma': 4.646888307676538, 'max_bin': 363}. Best is trial 10 with value: 0.5473365544016147.
Best trial: 31. Best value: 0.552707:  32%|███▏      | 32/100 [13:28<17:34, 15.51s/it, 808.99/3600 seconds]Trial  31: Precision=0.5447 | Recall=0.6198 | F1=0.5712 | F0.5=0.5527
[I 2025-06-12 15:15:00,741] Trial 31 finished with value: 0.5527067328903416 and parameters: {'n_estimators': 1816, 'max_depth': 11, 'learning_rate': 0.08318762169400208, 'subsample': 0.9747445700768947, 'colsample_bytree': 0.8482585848849901, 'colsample_bylevel': 0.6921613926016063, 'colsample_bynode': 0.9573872519911131, 'reg_alpha': 2.6597901572703125e-08, 'reg_lambda': 6.018239212121833e-06, 'min_child_weight': 10, 'gamma': 4.92976117882182, 'max_bin': 499}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  33%|███▎      | 33/100 [13:38<15:10, 13.60s/it, 818.13/3600 seconds]Trial  32: Precision=0.5449 | Recall=0.5855 | F1=0.5526 | F0.5=0.5446
[I 2025-06-12 15:15:09,878] Trial 32 finished with value: 0.5445993469972631 and parameters: {'n_estimators': 1564, 'max_depth': 10, 'learning_rate': 0.21539942941903265, 'subsample': 0.9955538511011638, 'colsample_bytree': 0.8491048368875511, 'colsample_bylevel': 0.8486539439878545, 'colsample_bynode': 0.9779510389196041, 'reg_alpha': 1.756370572997982e-08, 'reg_lambda': 9.484492734942522, 'min_child_weight': 9, 'gamma': 4.206821317280739, 'max_bin': 473}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  34%|███▍      | 34/100 [13:48<13:52, 12.61s/it, 828.43/3600 seconds]Trial  33: Precision=0.5296 | Recall=0.5087 | F1=0.5111 | F0.5=0.5197
[I 2025-06-12 15:15:20,182] Trial 33 finished with value: 0.5197000147396642 and parameters: {'n_estimators': 1629, 'max_depth': 10, 'learning_rate': 0.20438640839132063, 'subsample': 0.9956107313525514, 'colsample_bytree': 0.8449046947308698, 'colsample_bylevel': 0.8317004818792001, 'colsample_bynode': 0.9838788247270579, 'reg_alpha': 2.235553936097401e-08, 'reg_lambda': 5.009653871143045e-05, 'min_child_weight': 9, 'gamma': 4.0193048636901745, 'max_bin': 470}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  35%|███▌      | 35/100 [13:57<12:34, 11.61s/it, 837.70/3600 seconds]Trial  34: Precision=0.5367 | Recall=0.5767 | F1=0.5519 | F0.5=0.5417
[I 2025-06-12 15:15:29,449] Trial 34 finished with value: 0.5417242206903994 and parameters: {'n_estimators': 1512, 'max_depth': 14, 'learning_rate': 0.4324345944660587, 'subsample': 0.9569206785091909, 'colsample_bytree': 0.8496792540711392, 'colsample_bylevel': 0.7745271327033333, 'colsample_bynode': 0.9062915711506379, 'reg_alpha': 1.6132270277097248e-07, 'reg_lambda': 5.828138061838733, 'min_child_weight': 9, 'gamma': 4.986134336038922, 'max_bin': 467}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  36%|███▌      | 36/100 [14:08<12:15, 11.49s/it, 848.93/3600 seconds]Trial  35: Precision=0.5404 | Recall=0.5275 | F1=0.5244 | F0.5=0.5307
[I 2025-06-12 15:15:40,677] Trial 35 finished with value: 0.5306682650226393 and parameters: {'n_estimators': 1730, 'max_depth': 12, 'learning_rate': 0.30510248281005314, 'subsample': 0.925598187117271, 'colsample_bytree': 0.8273419933696505, 'colsample_bylevel': 0.8727708047827943, 'colsample_bynode': 0.9582453078299917, 'reg_alpha': 2.4996342977689448e-08, 'reg_lambda': 0.0014181387584406419, 'min_child_weight': 10, 'gamma': 4.345097614994303, 'max_bin': 445}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  37%|███▋      | 37/100 [14:16<10:46, 10.25s/it, 856.29/3600 seconds]Trial  36: Precision=0.5400 | Recall=0.5184 | F1=0.5218 | F0.5=0.5308
[I 2025-06-12 15:15:48,042] Trial 36 finished with value: 0.5308424300140967 and parameters: {'n_estimators': 993, 'max_depth': 11, 'learning_rate': 0.10076980640753007, 'subsample': 0.8696729696537576, 'colsample_bytree': 0.6840654773930064, 'colsample_bylevel': 0.9100222982935362, 'colsample_bynode': 0.8632032478556128, 'reg_alpha': 7.073631877612704e-06, 'reg_lambda': 1.3506690295628998e-05, 'min_child_weight': 8, 'gamma': 4.687413492163072, 'max_bin': 499}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  38%|███▊      | 38/100 [14:30<11:58, 11.59s/it, 870.99/3600 seconds]Trial  37: Precision=0.5408 | Recall=0.4954 | F1=0.5036 | F0.5=0.5209
[I 2025-06-12 15:16:02,739] Trial 37 finished with value: 0.5208870316739531 and parameters: {'n_estimators': 1883, 'max_depth': 14, 'learning_rate': 0.17885739375361678, 'subsample': 0.7723917796530066, 'colsample_bytree': 0.99859276180781, 'colsample_bylevel': 0.6601476369750394, 'colsample_bynode': 0.7716634527118463, 'reg_alpha': 9.984837371101858e-08, 'reg_lambda': 0.015972388264823594, 'min_child_weight': 10, 'gamma': 3.5909521249846827, 'max_bin': 476}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  39%|███▉      | 39/100 [14:59<17:01, 16.75s/it, 899.80/3600 seconds]Trial  38: Precision=0.5494 | Recall=0.5091 | F1=0.5137 | F0.5=0.5302
[I 2025-06-12 15:16:31,545] Trial 38 finished with value: 0.5302432748852364 and parameters: {'n_estimators': 1457, 'max_depth': 13, 'learning_rate': 0.010717398857983153, 'subsample': 0.9004672217949161, 'colsample_bytree': 0.9167208936736448, 'colsample_bylevel': 0.8241193354060564, 'colsample_bynode': 0.9115577472259978, 'reg_alpha': 2.2727839333244235e-06, 'reg_lambda': 0.00014484609672855704, 'min_child_weight': 7, 'gamma': 4.185667811658899, 'max_bin': 334}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 31. Best value: 0.552707:  40%|████      | 40/100 [15:09<14:30, 14.52s/it, 909.09/3600 seconds]Trial  39: Precision=0.5395 | Recall=0.4652 | F1=0.4826 | F0.5=0.5098
[I 2025-06-12 15:16:40,841] Trial 39 finished with value: 0.5098054231526453 and parameters: {'n_estimators': 207, 'max_depth': 7, 'learning_rate': 0.09946006792980613, 'subsample': 0.8232008317712567, 'colsample_bytree': 0.8608025974186068, 'colsample_bylevel': 0.7926972808404957, 'colsample_bynode': 0.9420681344891177, 'reg_alpha': 3.2702008178588526e-08, 'reg_lambda': 5.280339766413153e-06, 'min_child_weight': 9, 'gamma': 0.7561010317095747, 'max_bin': 295}. Best is trial 31 with value: 0.5527067328903416.
Best trial: 40. Best value: 0.565769:  41%|████      | 41/100 [15:14<11:41, 11.88s/it, 914.84/3600 seconds]Trial  40: Precision=0.5394 | Recall=0.7433 | F1=0.6154 | F0.5=0.5658
[I 2025-06-12 15:16:46,587] Trial 40 finished with value: 0.5657686140401919 and parameters: {'n_estimators': 840, 'max_depth': 10, 'learning_rate': 0.1622554136888379, 'subsample': 0.9473268865480898, 'colsample_bytree': 0.8173638013273287, 'colsample_bylevel': 0.7346008391077865, 'colsample_bynode': 0.9761007246217213, 'reg_alpha': 8.055925728983402, 'reg_lambda': 9.172233584927825e-08, 'min_child_weight': 10, 'gamma': 4.677404011585716, 'max_bin': 454}. Best is trial 40 with value: 0.5657686140401919.
Best trial: 40. Best value: 0.565769:  42%|████▏     | 42/100 [15:20<09:42, 10.05s/it, 920.60/3600 seconds]Trial  41: Precision=0.5408 | Recall=0.5778 | F1=0.5460 | F0.5=0.5395
[I 2025-06-12 15:16:52,348] Trial 41 finished with value: 0.5395409417292515 and parameters: {'n_estimators': 852, 'max_depth': 10, 'learning_rate': 0.16546978833462303, 'subsample': 0.9843116279426305, 'colsample_bytree': 0.8240668898146918, 'colsample_bylevel': 0.7309416702703986, 'colsample_bynode': 0.9793463400898459, 'reg_alpha': 0.15497359707933334, 'reg_lambda': 1.4272202783968739e-08, 'min_child_weight': 10, 'gamma': 4.677569127454802, 'max_bin': 453}. Best is trial 40 with value: 0.5657686140401919.
Best trial: 42. Best value: 0.56723:  43%|████▎     | 43/100 [15:23<07:33,  7.96s/it, 923.70/3600 seconds] Trial  42: Precision=0.5376 | Recall=0.7619 | F1=0.6222 | F0.5=0.5672
[I 2025-06-12 15:16:55,452] Trial 42 finished with value: 0.5672303869029487 and parameters: {'n_estimators': 544, 'max_depth': 10, 'learning_rate': 0.2938582603380786, 'subsample': 0.9552911410477546, 'colsample_bytree': 0.7991091655074166, 'colsample_bylevel': 0.7747986866913422, 'colsample_bynode': 0.6901392596118723, 'reg_alpha': 7.615371465435688, 'reg_lambda': 3.675517635889162e-08, 'min_child_weight': 10, 'gamma': 4.986366774672873, 'max_bin': 414}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  44%|████▍     | 44/100 [15:26<06:04,  6.51s/it, 926.81/3600 seconds]Trial  43: Precision=0.5439 | Recall=0.5473 | F1=0.5364 | F0.5=0.5377
[I 2025-06-12 15:16:58,562] Trial 43 finished with value: 0.5377398722831043 and parameters: {'n_estimators': 448, 'max_depth': 10, 'learning_rate': 0.2926681128379413, 'subsample': 0.9518657738813817, 'colsample_bytree': 0.7931845298036111, 'colsample_bylevel': 0.7746493420757486, 'colsample_bynode': 0.7178476013510283, 'reg_alpha': 3.603227201603901, 'reg_lambda': 9.591274845698437e-08, 'min_child_weight': 9, 'gamma': 4.591640603122615, 'max_bin': 409}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  45%|████▌     | 45/100 [15:30<05:18,  5.79s/it, 930.94/3600 seconds]Trial  44: Precision=0.5468 | Recall=0.6201 | F1=0.5756 | F0.5=0.5567
[I 2025-06-12 15:17:02,690] Trial 44 finished with value: 0.5567005613918015 and parameters: {'n_estimators': 696, 'max_depth': 8, 'learning_rate': 0.33010597254085583, 'subsample': 0.9262491655154511, 'colsample_bytree': 0.8917566000046583, 'colsample_bylevel': 0.733909258546673, 'colsample_bynode': 0.682895035796052, 'reg_alpha': 9.574576055143178, 'reg_lambda': 9.594424392618064e-08, 'min_child_weight': 10, 'gamma': 4.265817347555404, 'max_bin': 377}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  46%|████▌     | 46/100 [15:34<04:43,  5.25s/it, 934.92/3600 seconds]Trial  45: Precision=0.5375 | Recall=0.7127 | F1=0.6080 | F0.5=0.5629
[I 2025-06-12 15:17:06,672] Trial 45 finished with value: 0.562937404049217 and parameters: {'n_estimators': 560, 'max_depth': 7, 'learning_rate': 0.35279038615947583, 'subsample': 0.9200292554553168, 'colsample_bytree': 0.8118712562580529, 'colsample_bylevel': 0.7301305299518506, 'colsample_bynode': 0.6841349510390149, 'reg_alpha': 9.445944973830327, 'reg_lambda': 5.789608807751393e-08, 'min_child_weight': 10, 'gamma': 3.715684095646795, 'max_bin': 371}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  47%|████▋     | 47/100 [15:39<04:32,  5.14s/it, 939.81/3600 seconds]Trial  46: Precision=0.5444 | Recall=0.4381 | F1=0.4770 | F0.5=0.5125
[I 2025-06-12 15:17:11,561] Trial 46 finished with value: 0.5125286232083259 and parameters: {'n_estimators': 638, 'max_depth': 6, 'learning_rate': 0.47152627810536485, 'subsample': 0.9082315792591643, 'colsample_bytree': 0.894520124784577, 'colsample_bylevel': 0.7371893221446915, 'colsample_bynode': 0.6847139327777724, 'reg_alpha': 0.8511929903124492, 'reg_lambda': 4.246135459170971e-08, 'min_child_weight': 10, 'gamma': 3.753447232438254, 'max_bin': 371}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  48%|████▊     | 48/100 [15:42<03:54,  4.51s/it, 942.86/3600 seconds]Trial  47: Precision=0.5367 | Recall=0.6769 | F1=0.5900 | F0.5=0.5550
[I 2025-06-12 15:17:14,608] Trial 47 finished with value: 0.5550462146626065 and parameters: {'n_estimators': 397, 'max_depth': 7, 'learning_rate': 0.3835832313890989, 'subsample': 0.8698614883700949, 'colsample_bytree': 0.7594161287459923, 'colsample_bylevel': 0.7769794342908466, 'colsample_bynode': 0.6291521008583488, 'reg_alpha': 8.97097101146731, 'reg_lambda': 1.1193711565081403e-07, 'min_child_weight': 4, 'gamma': 3.2761879065736563, 'max_bin': 338}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  49%|████▉     | 49/100 [15:45<03:25,  4.03s/it, 945.75/3600 seconds]Trial  48: Precision=0.5376 | Recall=0.6301 | F1=0.5605 | F0.5=0.5412
[I 2025-06-12 15:17:17,494] Trial 48 finished with value: 0.5411880360287091 and parameters: {'n_estimators': 361, 'max_depth': 7, 'learning_rate': 0.3765605797705223, 'subsample': 0.8784224198276637, 'colsample_bytree': 0.6882680221161375, 'colsample_bylevel': 0.8084150039017949, 'colsample_bynode': 0.6289507652659951, 'reg_alpha': 7.975569774526406, 'reg_lambda': 9.068185296577855e-08, 'min_child_weight': 4, 'gamma': 2.952938886659701, 'max_bin': 337}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  50%|█████     | 50/100 [15:51<03:48,  4.57s/it, 951.60/3600 seconds]Trial  49: Precision=0.5265 | Recall=0.5138 | F1=0.5087 | F0.5=0.5161
[I 2025-06-12 15:17:23,350] Trial 49 finished with value: 0.5161056364919774 and parameters: {'n_estimators': 751, 'max_depth': 3, 'learning_rate': 0.33557275364803973, 'subsample': 0.8699738612508241, 'colsample_bytree': 0.7530917474938356, 'colsample_bylevel': 0.7767688906300976, 'colsample_bynode': 0.6033042870671139, 'reg_alpha': 0.20273854620931753, 'reg_lambda': 2.1112443079069013e-08, 'min_child_weight': 3, 'gamma': 3.218658106879926, 'max_bin': 317}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  51%|█████     | 51/100 [15:56<03:43,  4.56s/it, 956.12/3600 seconds]Trial  50: Precision=0.5385 | Recall=0.5653 | F1=0.5447 | F0.5=0.5392
[I 2025-06-12 15:17:27,866] Trial 50 finished with value: 0.5391953044786464 and parameters: {'n_estimators': 579, 'max_depth': 6, 'learning_rate': 0.3809549100792258, 'subsample': 0.7926477324599952, 'colsample_bytree': 0.7722180270609523, 'colsample_bylevel': 0.7997699479887511, 'colsample_bynode': 0.6809915376456998, 'reg_alpha': 7.108710255686432, 'reg_lambda': 1.2680926332142704e-07, 'min_child_weight': 5, 'gamma': 3.7976110896978064, 'max_bin': 353}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  52%|█████▏    | 52/100 [16:02<04:05,  5.12s/it, 962.56/3600 seconds]Trial  51: Precision=0.5406 | Recall=0.5187 | F1=0.5182 | F0.5=0.5280
[I 2025-06-12 15:17:34,306] Trial 51 finished with value: 0.5280208717463698 and parameters: {'n_estimators': 871, 'max_depth': 7, 'learning_rate': 0.25940108243562443, 'subsample': 0.9311369947846815, 'colsample_bytree': 0.8068672077998378, 'colsample_bylevel': 0.7099095194324501, 'colsample_bynode': 0.6601116122528707, 'reg_alpha': 2.524254653952177, 'reg_lambda': 8.729485642413214e-07, 'min_child_weight': 3, 'gamma': 2.4082120530165265, 'max_bin': 382}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  53%|█████▎    | 53/100 [16:06<03:45,  4.80s/it, 966.61/3600 seconds]Trial  52: Precision=0.5403 | Recall=0.4081 | F1=0.4386 | F0.5=0.4847
[I 2025-06-12 15:17:38,363] Trial 52 finished with value: 0.48468267218264066 and parameters: {'n_estimators': 422, 'max_depth': 5, 'learning_rate': 0.1252169766083488, 'subsample': 0.9125189667306356, 'colsample_bytree': 0.7224497019865433, 'colsample_bylevel': 0.76444848561412, 'colsample_bynode': 0.6984761943428841, 'reg_alpha': 0.33363598311290904, 'reg_lambda': 1.9404383059062544e-07, 'min_child_weight': 4, 'gamma': 3.3770684950251764, 'max_bin': 396}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  54%|█████▍    | 54/100 [16:13<04:07,  5.38s/it, 973.36/3600 seconds]Trial  53: Precision=0.5498 | Recall=0.4334 | F1=0.4483 | F0.5=0.4816
[I 2025-06-12 15:17:45,107] Trial 53 finished with value: 0.4815502222878437 and parameters: {'n_estimators': 752, 'max_depth': 8, 'learning_rate': 0.1766678428370133, 'subsample': 0.9495646806534406, 'colsample_bytree': 0.7819102939902971, 'colsample_bylevel': 0.738601480744785, 'colsample_bynode': 0.6352149563064649, 'reg_alpha': 0.07464744075638187, 'reg_lambda': 5.0170368669623673e-08, 'min_child_weight': 10, 'gamma': 2.6677489022075704, 'max_bin': 369}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  55%|█████▌    | 55/100 [16:16<03:38,  4.86s/it, 976.98/3600 seconds]Trial  54: Precision=0.5426 | Recall=0.5057 | F1=0.5081 | F0.5=0.5239
[I 2025-06-12 15:17:48,728] Trial 54 finished with value: 0.5239218591226606 and parameters: {'n_estimators': 522, 'max_depth': 8, 'learning_rate': 0.2762564614008492, 'subsample': 0.8598055628743251, 'colsample_bytree': 0.8304749504593194, 'colsample_bylevel': 0.7217159790733125, 'colsample_bynode': 0.7288633474138632, 'reg_alpha': 1.1949146122095802, 'reg_lambda': 2.4781183868467153e-08, 'min_child_weight': 6, 'gamma': 4.436170532432055, 'max_bin': 323}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  56%|█████▌    | 56/100 [16:18<02:54,  3.96s/it, 978.86/3600 seconds]Trial  55: Precision=0.5371 | Recall=0.5712 | F1=0.5464 | F0.5=0.5386
[I 2025-06-12 15:17:50,613] Trial 55 finished with value: 0.5386374258319301 and parameters: {'n_estimators': 233, 'max_depth': 6, 'learning_rate': 0.4854360429455281, 'subsample': 0.9207783716746454, 'colsample_bytree': 0.755694993636144, 'colsample_bylevel': 0.6994111992122172, 'colsample_bynode': 0.6685653833182013, 'reg_alpha': 3.6365902674819015, 'reg_lambda': 9.991363545182968e-07, 'min_child_weight': 5, 'gamma': 3.997271569061021, 'max_bin': 417}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  57%|█████▋    | 57/100 [16:25<03:23,  4.73s/it, 985.38/3600 seconds]Trial  56: Precision=0.5381 | Recall=0.5220 | F1=0.5231 | F0.5=0.5302
[I 2025-06-12 15:17:57,129] Trial 56 finished with value: 0.5302486180397857 and parameters: {'n_estimators': 918, 'max_depth': 9, 'learning_rate': 0.3442551849679718, 'subsample': 0.9027486048174467, 'colsample_bytree': 0.8097123985196139, 'colsample_bylevel': 0.7887100666501069, 'colsample_bynode': 0.6489658621964883, 'reg_alpha': 0.44859572158745464, 'reg_lambda': 3.2649366101547366e-08, 'min_child_weight': 10, 'gamma': 4.32826352652885, 'max_bin': 347}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  58%|█████▊    | 58/100 [16:29<03:09,  4.50s/it, 989.35/3600 seconds]Trial  57: Precision=0.5415 | Recall=0.7033 | F1=0.6004 | F0.5=0.5613
[I 2025-06-12 15:18:01,100] Trial 57 finished with value: 0.5613187863011683 and parameters: {'n_estimators': 665, 'max_depth': 7, 'learning_rate': 0.15090934276918974, 'subsample': 0.9341114085284519, 'colsample_bytree': 0.737367722480231, 'colsample_bylevel': 0.6978221650091054, 'colsample_bynode': 0.7550637721766501, 'reg_alpha': 7.779543925433442, 'reg_lambda': 3.2515309670862366e-07, 'min_child_weight': 9, 'gamma': 3.6743644729425684, 'max_bin': 390}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  59%|█████▉    | 59/100 [16:33<03:03,  4.48s/it, 993.78/3600 seconds]Trial  58: Precision=0.5424 | Recall=0.6886 | F1=0.5992 | F0.5=0.5625
[I 2025-06-12 15:18:05,527] Trial 58 finished with value: 0.562535297584065 and parameters: {'n_estimators': 633, 'max_depth': 7, 'learning_rate': 0.14233473448533315, 'subsample': 0.9355646972597608, 'colsample_bytree': 0.7001829143651069, 'colsample_bylevel': 0.7435240222651017, 'colsample_bynode': 0.7603883093365176, 'reg_alpha': 9.964976160192634, 'reg_lambda': 2.2832010902834984e-07, 'min_child_weight': 9, 'gamma': 3.6049860378960092, 'max_bin': 381}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  60%|██████    | 60/100 [16:40<03:24,  5.12s/it, 1000.41/3600 seconds]Trial  59: Precision=0.5348 | Recall=0.4938 | F1=0.4963 | F0.5=0.5137
[I 2025-06-12 15:18:12,155] Trial 59 finished with value: 0.5136577112581783 and parameters: {'n_estimators': 683, 'max_depth': 4, 'learning_rate': 0.1613069710982809, 'subsample': 0.723720478081262, 'colsample_bytree': 0.6131322858908977, 'colsample_bylevel': 0.6632043140093157, 'colsample_bynode': 0.764570283658172, 'reg_alpha': 1.6271540472063692, 'reg_lambda': 2.9493886595101855e-07, 'min_child_weight': 8, 'gamma': 3.60877502993062, 'max_bin': 393}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  61%|██████    | 61/100 [16:46<03:26,  5.29s/it, 1006.09/3600 seconds]Trial  60: Precision=0.5433 | Recall=0.6621 | F1=0.5854 | F0.5=0.5568
[I 2025-06-12 15:18:17,835] Trial 60 finished with value: 0.556767283942634 and parameters: {'n_estimators': 752, 'max_depth': 8, 'learning_rate': 0.13182437945974312, 'subsample': 0.9398820359354675, 'colsample_bytree': 0.6900795645672505, 'colsample_bylevel': 0.742667402513763, 'colsample_bynode': 0.7458565262852523, 'reg_alpha': 4.898881513011351, 'reg_lambda': 1.002594077741125e-08, 'min_child_weight': 9, 'gamma': 2.912585435622252, 'max_bin': 403}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  62%|██████▏   | 62/100 [16:50<03:08,  4.95s/it, 1010.25/3600 seconds]Trial  61: Precision=0.5398 | Recall=0.6082 | F1=0.5693 | F0.5=0.5506
[I 2025-06-12 15:18:22,003] Trial 61 finished with value: 0.5506429356766287 and parameters: {'n_estimators': 581, 'max_depth': 8, 'learning_rate': 0.1406678175419145, 'subsample': 0.9404525353969898, 'colsample_bytree': 0.7071901539716059, 'colsample_bylevel': 0.7442772053263754, 'colsample_bynode': 0.7561698030193705, 'reg_alpha': 4.157278900510569, 'reg_lambda': 1.1627434684920861e-08, 'min_child_weight': 9, 'gamma': 3.0482494112528675, 'max_bin': 403}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  63%|██████▎   | 63/100 [16:55<03:07,  5.07s/it, 1015.60/3600 seconds]Trial  62: Precision=0.5389 | Recall=0.7007 | F1=0.6020 | F0.5=0.5612
[I 2025-06-12 15:18:27,350] Trial 62 finished with value: 0.5612115252979184 and parameters: {'n_estimators': 771, 'max_depth': 7, 'learning_rate': 0.21389203779152252, 'subsample': 0.9610763460456629, 'colsample_bytree': 0.6540924237291258, 'colsample_bylevel': 0.7226565048594638, 'colsample_bynode': 0.7429147707694221, 'reg_alpha': 9.476557406893827, 'reg_lambda': 6.343224117427822e-08, 'min_child_weight': 9, 'gamma': 2.231415654732258, 'max_bin': 377}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  64%|██████▍   | 64/100 [17:02<03:24,  5.69s/it, 1022.74/3600 seconds]Trial  63: Precision=0.5504 | Recall=0.4893 | F1=0.5088 | F0.5=0.5302
[I 2025-06-12 15:18:34,486] Trial 63 finished with value: 0.5302111620249494 and parameters: {'n_estimators': 797, 'max_depth': 7, 'learning_rate': 0.21289515520387076, 'subsample': 0.9607927829277162, 'colsample_bytree': 0.6561087657506139, 'colsample_bylevel': 0.7568387590495396, 'colsample_bynode': 0.7426063758740388, 'reg_alpha': 1.7380018998851097, 'reg_lambda': 4.837895092625439e-08, 'min_child_weight': 9, 'gamma': 2.0149440182005756, 'max_bin': 357}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  65%|██████▌   | 65/100 [17:11<03:47,  6.50s/it, 1031.11/3600 seconds]Trial  64: Precision=0.5389 | Recall=0.4784 | F1=0.5002 | F0.5=0.5209
[I 2025-06-12 15:18:42,856] Trial 64 finished with value: 0.5209422546569485 and parameters: {'n_estimators': 1031, 'max_depth': 6, 'learning_rate': 0.1185810713984402, 'subsample': 0.9431095243806557, 'colsample_bytree': 0.6257372453457302, 'colsample_bylevel': 0.7211438493979149, 'colsample_bynode': 0.7913097342790076, 'reg_alpha': 0.7166699502192929, 'reg_lambda': 2.1157931829121715e-08, 'min_child_weight': 8, 'gamma': 2.146192877120734, 'max_bin': 434}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  66%|██████▌   | 66/100 [17:15<03:16,  5.77s/it, 1035.18/3600 seconds]Trial  65: Precision=0.5489 | Recall=0.5565 | F1=0.5372 | F0.5=0.5383
[I 2025-06-12 15:18:46,928] Trial 65 finished with value: 0.5382862539517533 and parameters: {'n_estimators': 511, 'max_depth': 5, 'learning_rate': 0.2389926863655854, 'subsample': 0.8948925937233445, 'colsample_bytree': 0.6744046953878168, 'colsample_bylevel': 0.7090844689002607, 'colsample_bynode': 0.7873317152603284, 'reg_alpha': 4.028368314176063, 'reg_lambda': 9.95325638648197e-07, 'min_child_weight': 8, 'gamma': 2.676598352165724, 'max_bin': 388}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  67%|██████▋   | 67/100 [17:22<03:21,  6.10s/it, 1042.04/3600 seconds]Trial  66: Precision=0.5402 | Recall=0.4568 | F1=0.4800 | F0.5=0.5089
[I 2025-06-12 15:18:53,793] Trial 66 finished with value: 0.5088975477899322 and parameters: {'n_estimators': 630, 'max_depth': 9, 'learning_rate': 0.14567423690693496, 'subsample': 0.9679817020393505, 'colsample_bytree': 0.6340736453856228, 'colsample_bylevel': 0.7637796313803028, 'colsample_bynode': 0.832783913644113, 'reg_alpha': 2.0321293464517134, 'reg_lambda': 2.079177904331061e-07, 'min_child_weight': 9, 'gamma': 1.4898673875389354, 'max_bin': 414}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  68%|██████▊   | 68/100 [17:27<03:04,  5.76s/it, 1047.00/3600 seconds]Trial  67: Precision=0.5317 | Recall=0.6497 | F1=0.5769 | F0.5=0.5470
[I 2025-06-12 15:18:58,754] Trial 67 finished with value: 0.5469682645376396 and parameters: {'n_estimators': 779, 'max_depth': 7, 'learning_rate': 0.20612249259007517, 'subsample': 0.9418789966337705, 'colsample_bytree': 0.7036181937579074, 'colsample_bylevel': 0.6751063252495746, 'colsample_bynode': 0.7121207532589455, 'reg_alpha': 5.255512381797356, 'reg_lambda': 5.970852717299491e-07, 'min_child_weight': 9, 'gamma': 2.824753569670697, 'max_bin': 401}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  69%|██████▉   | 69/100 [17:34<03:12,  6.22s/it, 1054.29/3600 seconds]Trial  68: Precision=0.5400 | Recall=0.4937 | F1=0.5015 | F0.5=0.5188
[I 2025-06-12 15:19:06,043] Trial 68 finished with value: 0.518766588699581 and parameters: {'n_estimators': 939, 'max_depth': 8, 'learning_rate': 0.19427300463430647, 'subsample': 0.9877335139329124, 'colsample_bytree': 0.7394840570485177, 'colsample_bylevel': 0.6979207050303272, 'colsample_bynode': 0.7504732292455413, 'reg_alpha': 0.06795642680327973, 'reg_lambda': 5.488810392831189e-08, 'min_child_weight': 8, 'gamma': 1.8064812897515168, 'max_bin': 423}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  70%|███████   | 70/100 [17:42<03:27,  6.93s/it, 1062.88/3600 seconds]Trial  69: Precision=0.5494 | Recall=0.5264 | F1=0.5284 | F0.5=0.5385
[I 2025-06-12 15:19:14,630] Trial 69 finished with value: 0.5384669768901491 and parameters: {'n_estimators': 1121, 'max_depth': 5, 'learning_rate': 0.10796486766269561, 'subsample': 0.9590583943061525, 'colsample_bytree': 0.6682368930617869, 'colsample_bylevel': 0.7254446337679731, 'colsample_bynode': 0.7265649830834799, 'reg_alpha': 1.0681039019372154, 'reg_lambda': 1.1456630797357442e-08, 'min_child_weight': 9, 'gamma': 2.210004787877786, 'max_bin': 381}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  71%|███████   | 71/100 [17:52<03:45,  7.77s/it, 1072.63/3600 seconds]Trial  70: Precision=0.5465 | Recall=0.5427 | F1=0.5264 | F0.5=0.5323
[I 2025-06-12 15:19:24,377] Trial 70 finished with value: 0.5322592900236144 and parameters: {'n_estimators': 335, 'max_depth': 6, 'learning_rate': 0.05970401306526717, 'subsample': 0.9156203240451252, 'colsample_bytree': 0.6945914485907692, 'colsample_bylevel': 0.7484975550687348, 'colsample_bynode': 0.7388492799903759, 'reg_alpha': 2.131127805426838, 'reg_lambda': 2.6595916984903582e-08, 'min_child_weight': 10, 'gamma': 1.2738536179071949, 'max_bin': 374}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  72%|███████▏  | 72/100 [17:57<03:12,  6.87s/it, 1077.38/3600 seconds]Trial  71: Precision=0.5399 | Recall=0.7154 | F1=0.6086 | F0.5=0.5643
[I 2025-06-12 15:19:29,130] Trial 71 finished with value: 0.5643147228567745 and parameters: {'n_estimators': 700, 'max_depth': 8, 'learning_rate': 0.15042617692527707, 'subsample': 0.9267515243832022, 'colsample_bytree': 0.6452651461782604, 'colsample_bylevel': 0.7344403402664156, 'colsample_bynode': 0.7041068475149477, 'reg_alpha': 9.92018995881969, 'reg_lambda': 1.0183526825789151e-07, 'min_child_weight': 10, 'gamma': 3.726291815525026, 'max_bin': 361}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  73%|███████▎  | 73/100 [18:02<02:52,  6.39s/it, 1082.65/3600 seconds]Trial  72: Precision=0.5435 | Recall=0.6745 | F1=0.5889 | F0.5=0.5573
[I 2025-06-12 15:19:34,400] Trial 72 finished with value: 0.5573451480700019 and parameters: {'n_estimators': 719, 'max_depth': 7, 'learning_rate': 0.1422832272180706, 'subsample': 0.9334771192052745, 'colsample_bytree': 0.656432617968631, 'colsample_bylevel': 0.7185936948203601, 'colsample_bynode': 0.7108750963469327, 'reg_alpha': 5.59135227955162, 'reg_lambda': 2.0586571252897087e-07, 'min_child_weight': 10, 'gamma': 3.7206994041041486, 'max_bin': 389}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  74%|███████▍  | 74/100 [18:07<02:36,  6.02s/it, 1087.80/3600 seconds]Trial  73: Precision=0.5396 | Recall=0.7160 | F1=0.6076 | F0.5=0.5636
[I 2025-06-12 15:19:39,552] Trial 73 finished with value: 0.5635927318291845 and parameters: {'n_estimators': 471, 'max_depth': 7, 'learning_rate': 0.09248652390485813, 'subsample': 0.9277440686405946, 'colsample_bytree': 0.6418730548234576, 'colsample_bylevel': 0.7153170949361594, 'colsample_bynode': 0.7023794708964379, 'reg_alpha': 8.762735300138617, 'reg_lambda': 3.7900219791324744e-07, 'min_child_weight': 10, 'gamma': 3.710966971885814, 'max_bin': 388}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  75%|███████▌  | 75/100 [18:12<02:20,  5.64s/it, 1092.56/3600 seconds]Trial  74: Precision=0.5367 | Recall=0.7655 | F1=0.6225 | F0.5=0.5667
[I 2025-06-12 15:19:44,305] Trial 74 finished with value: 0.5666896717282883 and parameters: {'n_estimators': 628, 'max_depth': 9, 'learning_rate': 0.09090884711890766, 'subsample': 0.9674261225545349, 'colsample_bytree': 0.6345810015758323, 'colsample_bylevel': 0.6945053798561943, 'colsample_bynode': 0.6939879381165214, 'reg_alpha': 9.71587685704779, 'reg_lambda': 1.8553487182522462e-06, 'min_child_weight': 10, 'gamma': 3.9559533491009664, 'max_bin': 360}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  76%|███████▌  | 76/100 [18:18<02:17,  5.74s/it, 1098.54/3600 seconds]Trial  75: Precision=0.5545 | Recall=0.5643 | F1=0.5399 | F0.5=0.5413
[I 2025-06-12 15:19:50,286] Trial 75 finished with value: 0.541281012712467 and parameters: {'n_estimators': 491, 'max_depth': 9, 'learning_rate': 0.07005345511204292, 'subsample': 0.708709913591598, 'colsample_bytree': 0.6389657376308308, 'colsample_bylevel': 0.6793059313938137, 'colsample_bynode': 0.6695452596067984, 'reg_alpha': 2.65650009419808, 'reg_lambda': 1.8551248147451623e-06, 'min_child_weight': 10, 'gamma': 4.025897044245642, 'max_bin': 361}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  77%|███████▋  | 77/100 [18:24<02:16,  5.95s/it, 1104.96/3600 seconds]Trial  76: Precision=0.5497 | Recall=0.4809 | F1=0.4982 | F0.5=0.5234
[I 2025-06-12 15:19:56,713] Trial 76 finished with value: 0.5233909209078785 and parameters: {'n_estimators': 586, 'max_depth': 10, 'learning_rate': 0.09361405208411805, 'subsample': 0.8900239009361034, 'colsample_bytree': 0.6065995396799174, 'colsample_bylevel': 0.6502275733953901, 'colsample_bynode': 0.6946839893417436, 'reg_alpha': 0.0018449558306787343, 'reg_lambda': 4.633223730095784e-07, 'min_child_weight': 10, 'gamma': 3.447876128646352, 'max_bin': 347}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  78%|███████▊  | 78/100 [18:30<02:08,  5.83s/it, 1110.52/3600 seconds]Trial  77: Precision=0.5416 | Recall=0.5782 | F1=0.5539 | F0.5=0.5451
[I 2025-06-12 15:20:02,268] Trial 77 finished with value: 0.545148367435148 and parameters: {'n_estimators': 635, 'max_depth': 9, 'learning_rate': 0.0872889426646259, 'subsample': 0.9228715122804264, 'colsample_bytree': 0.6404335782076407, 'colsample_bylevel': 0.6918829887545697, 'colsample_bynode': 0.6980027067128934, 'reg_alpha': 0.33046413267593705, 'reg_lambda': 1.4912762190578448e-06, 'min_child_weight': 10, 'gamma': 3.9103052082347167, 'max_bin': 367}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  79%|███████▉  | 79/100 [18:34<01:48,  5.19s/it, 1114.21/3600 seconds]Trial  78: Precision=0.5496 | Recall=0.5737 | F1=0.5556 | F0.5=0.5505
[I 2025-06-12 15:20:05,963] Trial 78 finished with value: 0.5505158234073997 and parameters: {'n_estimators': 466, 'max_depth': 8, 'learning_rate': 0.11493422745759783, 'subsample': 0.9662008532829057, 'colsample_bytree': 0.6242736047907912, 'colsample_bylevel': 0.6976059506386784, 'colsample_bynode': 0.7825931767927549, 'reg_alpha': 0.5210116187083742, 'reg_lambda': 3.501107559785428e-07, 'min_child_weight': 10, 'gamma': 3.6594356133365893, 'max_bin': 363}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  80%|████████  | 80/100 [18:36<01:29,  4.45s/it, 1116.95/3600 seconds]Trial  79: Precision=0.5403 | Recall=0.6248 | F1=0.5711 | F0.5=0.5499
[I 2025-06-12 15:20:08,704] Trial 79 finished with value: 0.5498967110254916 and parameters: {'n_estimators': 325, 'max_depth': 9, 'learning_rate': 0.06671983179375827, 'subsample': 0.9884646717297285, 'colsample_bytree': 0.6011493963829208, 'colsample_bylevel': 0.7837970773388053, 'colsample_bynode': 0.7008522105761563, 'reg_alpha': 1.1807445875629765, 'reg_lambda': 1.335827038274954e-07, 'min_child_weight': 10, 'gamma': 4.7979562081259965, 'max_bin': 395}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  81%|████████  | 81/100 [18:40<01:20,  4.25s/it, 1120.74/3600 seconds]Trial  80: Precision=0.5409 | Recall=0.7296 | F1=0.6136 | F0.5=0.5666
[I 2025-06-12 15:20:12,493] Trial 80 finished with value: 0.5665650577536075 and parameters: {'n_estimators': 564, 'max_depth': 6, 'learning_rate': 0.14900670128283602, 'subsample': 0.9516365846541924, 'colsample_bytree': 0.7319111406276745, 'colsample_bylevel': 0.6343671357779904, 'colsample_bynode': 0.8231422622509429, 'reg_alpha': 9.906449599594751, 'reg_lambda': 2.7615657713201837e-07, 'min_child_weight': 9, 'gamma': 3.3627998724833428, 'max_bin': 351}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  82%|████████▏ | 82/100 [18:44<01:16,  4.25s/it, 1124.98/3600 seconds]Trial  81: Precision=0.5428 | Recall=0.6128 | F1=0.5669 | F0.5=0.5501
[I 2025-06-12 15:20:16,724] Trial 81 finished with value: 0.5501121267465708 and parameters: {'n_estimators': 555, 'max_depth': 6, 'learning_rate': 0.16085832599370473, 'subsample': 0.9540297937918986, 'colsample_bytree': 0.7153647222721817, 'colsample_bylevel': 0.6321052473995208, 'colsample_bynode': 0.6757481457650518, 'reg_alpha': 3.2225132864794848, 'reg_lambda': 3.6129891630313387e-06, 'min_child_weight': 9, 'gamma': 3.479367611454905, 'max_bin': 353}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  83%|████████▎ | 83/100 [18:49<01:13,  4.32s/it, 1129.45/3600 seconds]Trial  82: Precision=0.5439 | Recall=0.6925 | F1=0.5991 | F0.5=0.5628
[I 2025-06-12 15:20:21,203] Trial 82 finished with value: 0.5627822788498229 and parameters: {'n_estimators': 625, 'max_depth': 7, 'learning_rate': 0.1847245188605124, 'subsample': 0.9760808448823027, 'colsample_bytree': 0.6766470467049517, 'colsample_bylevel': 0.6141190285101056, 'colsample_bynode': 0.6907633549522333, 'reg_alpha': 5.895954942576608, 'reg_lambda': 5.384767344934897e-07, 'min_child_weight': 10, 'gamma': 3.306819693716742, 'max_bin': 345}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  84%|████████▍ | 84/100 [18:54<01:11,  4.48s/it, 1134.30/3600 seconds]Trial  83: Precision=0.5403 | Recall=0.4819 | F1=0.4986 | F0.5=0.5193
[I 2025-06-12 15:20:26,052] Trial 83 finished with value: 0.5193259043825437 and parameters: {'n_estimators': 616, 'max_depth': 6, 'learning_rate': 0.1841673115717723, 'subsample': 0.9826442002618643, 'colsample_bytree': 0.6691889819070597, 'colsample_bylevel': 0.6202965068155406, 'colsample_bynode': 0.6890513218638574, 'reg_alpha': 0.00016652153176108424, 'reg_lambda': 7.688731955622763e-07, 'min_child_weight': 10, 'gamma': 3.310921683895967, 'max_bin': 322}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  85%|████████▌ | 85/100 [19:00<01:13,  4.92s/it, 1140.24/3600 seconds]Trial  84: Precision=0.5435 | Recall=0.5111 | F1=0.5058 | F0.5=0.5204
[I 2025-06-12 15:20:31,993] Trial 84 finished with value: 0.5204042283429335 and parameters: {'n_estimators': 833, 'max_depth': 5, 'learning_rate': 0.2665659676588889, 'subsample': 0.9730385345005627, 'colsample_bytree': 0.676038393297929, 'colsample_bylevel': 0.608680318582079, 'colsample_bynode': 0.6522304431604289, 'reg_alpha': 2.2815470566854144, 'reg_lambda': 1.8113834566025482e-07, 'min_child_weight': 10, 'gamma': 3.488920844220839, 'max_bin': 347}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  86%|████████▌ | 86/100 [19:04<01:07,  4.80s/it, 1144.76/3600 seconds]Trial  85: Precision=0.5401 | Recall=0.7242 | F1=0.6087 | F0.5=0.5638
[I 2025-06-12 15:20:36,506] Trial 85 finished with value: 0.5637957978302711 and parameters: {'n_estimators': 531, 'max_depth': 10, 'learning_rate': 0.11220793743930033, 'subsample': 0.9485163349469341, 'colsample_bytree': 0.6430287475084236, 'colsample_bylevel': 0.6295309397985598, 'colsample_bynode': 0.7189045536743576, 'reg_alpha': 6.127766451325177, 'reg_lambda': 7.536080963921503e-08, 'min_child_weight': 10, 'gamma': 3.122839846103602, 'max_bin': 331}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  87%|████████▋ | 87/100 [19:07<00:54,  4.22s/it, 1147.62/3600 seconds]Trial  86: Precision=0.5409 | Recall=0.6738 | F1=0.5888 | F0.5=0.5566
[I 2025-06-12 15:20:39,369] Trial 86 finished with value: 0.5565819867141195 and parameters: {'n_estimators': 259, 'max_depth': 10, 'learning_rate': 0.10699679660701084, 'subsample': 0.9463980142559242, 'colsample_bytree': 0.6472490704354784, 'colsample_bylevel': 0.6158441023483585, 'colsample_bynode': 0.7088624265630927, 'reg_alpha': 5.059974943861212, 'reg_lambda': 4.487713069292179e-07, 'min_child_weight': 10, 'gamma': 3.1112539303516336, 'max_bin': 328}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  88%|████████▊ | 88/100 [19:11<00:48,  4.08s/it, 1151.37/3600 seconds]Trial  87: Precision=0.5465 | Recall=0.6058 | F1=0.5645 | F0.5=0.5505
[I 2025-06-12 15:20:43,119] Trial 87 finished with value: 0.550477155349715 and parameters: {'n_estimators': 405, 'max_depth': 11, 'learning_rate': 0.09057760485397731, 'subsample': 0.992811491941112, 'colsample_bytree': 0.6189436179425665, 'colsample_bylevel': 0.6012025145690316, 'colsample_bynode': 0.6664052186751555, 'reg_alpha': 1.3657341512854375, 'reg_lambda': 6.978382630580399e-08, 'min_child_weight': 10, 'gamma': 4.08143796102492, 'max_bin': 302}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  89%|████████▉ | 89/100 [19:14<00:41,  3.78s/it, 1154.46/3600 seconds]Trial  88: Precision=0.5359 | Recall=0.6856 | F1=0.5927 | F0.5=0.5555
[I 2025-06-12 15:20:46,213] Trial 88 finished with value: 0.555491342632936 and parameters: {'n_estimators': 544, 'max_depth': 10, 'learning_rate': 0.23894576133669004, 'subsample': 0.9767196201743018, 'colsample_bytree': 0.6454298627728279, 'colsample_bylevel': 0.6321385578902005, 'colsample_bynode': 0.7207241206624618, 'reg_alpha': 5.198698650391352, 'reg_lambda': 3.422239493826272e-08, 'min_child_weight': 10, 'gamma': 3.847692795607733, 'max_bin': 344}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  90%|█████████ | 90/100 [19:18<00:37,  3.77s/it, 1158.21/3600 seconds]Trial  89: Precision=0.5456 | Recall=0.5794 | F1=0.5489 | F0.5=0.5426
[I 2025-06-12 15:20:49,957] Trial 89 finished with value: 0.5425884226480184 and parameters: {'n_estimators': 478, 'max_depth': 10, 'learning_rate': 0.12967998463750788, 'subsample': 0.9545932349521901, 'colsample_bytree': 0.6279077548767724, 'colsample_bylevel': 0.6487128911286274, 'colsample_bynode': 0.7048465821353418, 'reg_alpha': 0.7525965436107802, 'reg_lambda': 1.2392010967236862e-06, 'min_child_weight': 10, 'gamma': 4.541452368225612, 'max_bin': 332}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  91%|█████████ | 91/100 [19:24<00:41,  4.62s/it, 1164.81/3600 seconds]Trial  90: Precision=0.5433 | Recall=0.5862 | F1=0.5540 | F0.5=0.5446
[I 2025-06-12 15:20:56,563] Trial 90 finished with value: 0.5445646857594629 and parameters: {'n_estimators': 888, 'max_depth': 8, 'learning_rate': 0.08160977514254299, 'subsample': 0.9051217891996255, 'colsample_bytree': 0.8195815314969782, 'colsample_bylevel': 0.6402231824784484, 'colsample_bynode': 0.6384085313767873, 'reg_alpha': 2.6616305823856488, 'reg_lambda': 1.2308109381230847e-07, 'min_child_weight': 10, 'gamma': 3.359774169798674, 'max_bin': 313}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  92%|█████████▏| 92/100 [19:29<00:37,  4.70s/it, 1169.68/3600 seconds]Trial  91: Precision=0.5436 | Recall=0.7037 | F1=0.6042 | F0.5=0.5645
[I 2025-06-12 15:21:01,430] Trial 91 finished with value: 0.5644644769462357 and parameters: {'n_estimators': 711, 'max_depth': 7, 'learning_rate': 0.17670287851936947, 'subsample': 0.9159138512012986, 'colsample_bytree': 0.6608830413999812, 'colsample_bylevel': 0.6213227169698713, 'colsample_bynode': 0.7298064049552108, 'reg_alpha': 8.189662464921948, 'reg_lambda': 2.5937859612401417e-07, 'min_child_weight': 9, 'gamma': 3.901790871144497, 'max_bin': 256}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  93%|█████████▎| 93/100 [19:35<00:34,  4.89s/it, 1175.03/3600 seconds]Trial  92: Precision=0.5431 | Recall=0.6745 | F1=0.5949 | F0.5=0.5613
[I 2025-06-12 15:21:06,775] Trial 92 finished with value: 0.5612842179738967 and parameters: {'n_estimators': 714, 'max_depth': 8, 'learning_rate': 0.17333268902955115, 'subsample': 0.9173475121588792, 'colsample_bytree': 0.6621241438727542, 'colsample_bylevel': 0.6212096384391257, 'colsample_bynode': 0.734194603949288, 'reg_alpha': 6.105238288325974, 'reg_lambda': 6.761764610475351e-08, 'min_child_weight': 10, 'gamma': 3.1544239952024826, 'max_bin': 260}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  94%|█████████▍| 94/100 [19:39<00:27,  4.64s/it, 1179.08/3600 seconds]Trial  93: Precision=0.5400 | Recall=0.5942 | F1=0.5606 | F0.5=0.5467
[I 2025-06-12 15:21:10,831] Trial 93 finished with value: 0.5467217366432047 and parameters: {'n_estimators': 536, 'max_depth': 9, 'learning_rate': 0.3017464704600499, 'subsample': 0.9265547123531434, 'colsample_bytree': 0.837546089257487, 'colsample_bylevel': 0.669367581263933, 'colsample_bynode': 0.6904365971633635, 'reg_alpha': 3.3766102654349237, 'reg_lambda': 6.263548332611771e-07, 'min_child_weight': 9, 'gamma': 3.969875256456028, 'max_bin': 286}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  95%|█████████▌| 95/100 [19:43<00:22,  4.59s/it, 1183.55/3600 seconds]Trial  94: Precision=0.5379 | Recall=0.7601 | F1=0.6211 | F0.5=0.5670
[I 2025-06-12 15:21:15,297] Trial 94 finished with value: 0.566968506597713 and parameters: {'n_estimators': 608, 'max_depth': 6, 'learning_rate': 0.11173617179594919, 'subsample': 0.9645911793225247, 'colsample_bytree': 0.6819910234781379, 'colsample_bylevel': 0.6548357000765006, 'colsample_bynode': 0.6762872555624292, 'reg_alpha': 9.748702379111164, 'reg_lambda': 1.4540818565090677e-07, 'min_child_weight': 10, 'gamma': 3.822133878533465, 'max_bin': 283}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  96%|█████████▌| 96/100 [19:47<00:17,  4.43s/it, 1187.62/3600 seconds]Trial  95: Precision=0.5437 | Recall=0.6890 | F1=0.5985 | F0.5=0.5623
[I 2025-06-12 15:21:19,373] Trial 95 finished with value: 0.5622917417248257 and parameters: {'n_estimators': 435, 'max_depth': 6, 'learning_rate': 0.11396244935635029, 'subsample': 0.6071665438789435, 'colsample_bytree': 0.7972716256375797, 'colsample_bylevel': 0.6533940817693582, 'colsample_bynode': 0.6792638282387041, 'reg_alpha': 8.70709378025926, 'reg_lambda': 1.713300102548607e-08, 'min_child_weight': 10, 'gamma': 3.7445353598132836, 'max_bin': 279}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  97%|█████████▋| 97/100 [19:52<00:13,  4.66s/it, 1192.82/3600 seconds]Trial  96: Precision=0.5431 | Recall=0.4950 | F1=0.5029 | F0.5=0.5212
[I 2025-06-12 15:21:24,567] Trial 96 finished with value: 0.5211786224662751 and parameters: {'n_estimators': 585, 'max_depth': 11, 'learning_rate': 0.12740575851826347, 'subsample': 0.8817685807661472, 'colsample_bytree': 0.632725022904053, 'colsample_bylevel': 0.6578095925122986, 'colsample_bynode': 0.7234126819199523, 'reg_alpha': 1.56052559590487, 'reg_lambda': 8.39200754634072e-08, 'min_child_weight': 9, 'gamma': 4.214402094727458, 'max_bin': 265}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  98%|█████████▊| 98/100 [19:57<00:09,  4.71s/it, 1197.63/3600 seconds]Trial  97: Precision=0.5415 | Recall=0.6685 | F1=0.5901 | F0.5=0.5584
[I 2025-06-12 15:21:29,379] Trial 97 finished with value: 0.5584349793200943 and parameters: {'n_estimators': 678, 'max_depth': 5, 'learning_rate': 0.062426157492642666, 'subsample': 0.9469861373613607, 'colsample_bytree': 0.7851185773652151, 'colsample_bylevel': 0.6417654835359158, 'colsample_bynode': 0.6603487836123665, 'reg_alpha': 3.551646851404558, 'reg_lambda': 3.913598496966698e-08, 'min_child_weight': 10, 'gamma': 3.82444515807558, 'max_bin': 271}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723:  99%|█████████▉| 99/100 [20:00<00:04,  4.30s/it, 1200.97/3600 seconds]Trial  98: Precision=0.5384 | Recall=0.7242 | F1=0.6081 | F0.5=0.5626
[I 2025-06-12 15:21:32,721] Trial 98 finished with value: 0.56264223988349 and parameters: {'n_estimators': 387, 'max_depth': 6, 'learning_rate': 0.05274842051120016, 'subsample': 0.896539589307603, 'colsample_bytree': 0.6145279816072757, 'colsample_bylevel': 0.624053323079727, 'colsample_bynode': 0.8319486468526521, 'reg_alpha': 9.686518910883882, 'reg_lambda': 1.2681845716288595e-07, 'min_child_weight': 9, 'gamma': 3.8982557090126146, 'max_bin': 358}. Best is trial 42 with value: 0.5672303869029487.
Best trial: 42. Best value: 0.56723: 100%|██████████| 100/100 [20:07<00:00, 12.08s/it, 1207.55/3600 seconds]Trial  99: Precision=0.5385 | Recall=0.5011 | F1=0.5061 | F0.5=0.5210
[I 2025-06-12 15:21:39,303] Trial 99 finished with value: 0.5210426457300523 and parameters: {'n_estimators': 817, 'max_depth': 10, 'learning_rate': 0.07510770874962336, 'subsample': 0.9676230464300466, 'colsample_bytree': 0.7688464590489239, 'colsample_bylevel': 0.6380566223441241, 'colsample_bynode': 0.6731368198425041, 'reg_alpha': 0.021670317002119688, 'reg_lambda': 2.66504636593889e-07, 'min_child_weight': 1, 'gamma': 4.434956228306662, 'max_bin': 291}. Best is trial 42 with value: 0.5672303869029487.
\nOptimization completed!
Best F0.5 score: 0.567230
Best parameters: {'n_estimators': 544, 'max_depth': 10, 'learning_rate': 0.2938582603380786, 'subsample': 0.9552911410477546, 'colsample_bytree': 0.7991091655074166, 'colsample_bylevel': 0.7747986866913422, 'colsample_bynode': 0.6901392596118723, 'reg_alpha': 7.615371465435688, 'reg_lambda': 3.675517635889162e-08, 'min_child_weight': 10, 'gamma': 4.986366774672873, 'max_bin': 414}
\nTraining final model with best F0.5 parameters...

"""

# ─── Run Parser ───
df_results = parse_xgboost_output_from_text(output_text)
print(df_results.head(10))  # Show first 10 results


   index  precision  recall      f1    f0.5     value  n_estimators  \
0      0     0.5372  0.5152  0.5193  0.5282  0.528217         812.0   
1      1     0.5404  0.4245  0.4492  0.4907  0.490670        1682.0   
2      2     0.5473  0.5291  0.5250  0.5339  0.533852         966.0   
3      3     0.5393  0.4882  0.4959  0.5165  0.516514         679.0   
4      4     0.5345  0.4684  0.4848  0.5090  0.508954        1139.0   
5      5     0.5338  0.5485  0.5292  0.5285  0.528493         838.0   
6      6     0.5380  0.4559  0.4752  0.5041  0.504087         110.0   
7      7     0.5390  0.4863  0.5025  0.5214  0.521403         691.0   
8      8     0.5438  0.4577  0.4754  0.5070  0.507038        1093.0   
9      9     0.5524  0.4606  0.4804  0.5125  0.512546         534.0   

   max_depth  learning_rate  subsample  colsample_bytree  colsample_bylevel  \
0       15.0       0.145532   0.839463          0.662407           0.662398   
1        5.0       0.011551   0.673362          0.721697    

In [10]:
!pip install -U xgboost



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# =============================================================
#  XGBoost Final Trainer  —  v2-compatible, leakage-free (FIXED)
# =============================================================
import os, warnings, joblib
from datetime import datetime
import logging

import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.callback import EarlyStopping

from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report,
                             confusion_matrix, roc_curve,
                             precision_recall_curve, average_precision_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

# Set up logging for better debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ──────────────────────────────────────────────────────────────
#  CONFIG  (update paths / params as needed)
# ──────────────────────────────────────────────────────────────
DATA_PATH     = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"
TARGET_COLUMN = "target"
DROP_COLUMNS  = [
    'open', 'high', 'low',
    'high_low', 'high_close', 'low_close', 'typical_price',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3', 
    'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'momentum_alignment',
  
]                 # e.g. ["timestamp"]
TEST_FRAC     = 0.20
RANDOM_SEED   = 42

# Optimal hyper-parameters found in Optuna search
BEST_PARAMS = {
    "n_estimators":        738,
    "max_depth":           3,
    "learning_rate":       0.010742242818480038,
    "subsample":           0.7288003215611947,
    "colsample_bytree":    0.9716424763198493,
    "colsample_bylevel":   0.9399534681398528,
    "reg_alpha":           0.012276608615353355,
    "reg_lambda":          0.0014806031253539519,
    "min_child_weight":    1,
    "gamma":               0.5640314445201723,
}
EARLY_STOP_ROUNDS = 100


# ──────────────────────────────────────────────────────────────
#  TRAINER CLASS
# ──────────────────────────────────────────────────────────────
class XGBoostFinalTrainer:
    def __init__(self,
                 data_path:      str,
                 target_column:  str,
                 drop_columns:   list = None,
                 test_frac:      float = 0.2,
                 random_state:   int   = 42):

        self.data_path      = data_path
        self.target_column  = target_column
        self.drop_columns   = drop_columns or []
        self.test_frac      = test_frac
        self.random_state   = random_state

        # populated later
        self.is_binary      = False
        self.label_encoders = {}
        self.X_train = self.X_test = self.y_train = self.y_test = None
        self.model   = None

    # ─── DATA ────────────────────────────────────────────────
    def load_and_preprocess(self):
        try:
            logger.info("📥 Loading data from: %s", self.data_path)
            
            # Check if file exists
            if not os.path.exists(self.data_path):
                raise FileNotFoundError(f"Data file not found: {self.data_path}")
            
            df = pd.read_csv(self.data_path)
            logger.info(f"Loaded data shape: {df.shape}")
            
            if df.empty:
                raise ValueError("Loaded dataframe is empty")

            if self.target_column not in df.columns:
                available_cols = list(df.columns)
                raise KeyError(f"Target column '{self.target_column}' not found. Available columns: {available_cols}")

            # Check for sufficient data
            if len(df) < 100:
                raise ValueError(f"Insufficient data: only {len(df)} rows. Need at least 100 rows for reliable training.")

            y = df[self.target_column].copy()
            X = df.drop(columns=[col for col in self.drop_columns + [self.target_column] if col in df.columns])

            logger.info(f"Features shape: {X.shape}, Target shape: {y.shape}")
            logger.info(f"Target value counts:\n{y.value_counts()}")

            # Handle missing values more robustly
            logger.info("Handling missing values...")
            for col in X.columns:
                missing_count = X[col].isna().sum()
                if missing_count > 0:
                    logger.info(f"Column '{col}' has {missing_count} missing values")
                    
                if X[col].dtype in ("object", "category"):
                    mode_val = X[col].mode()
                    fill_val = mode_val.iloc[0] if len(mode_val) > 0 else "unknown"
                    X[col] = X[col].fillna(fill_val)
                else:
                    X[col] = X[col].fillna(X[col].median())

            # Handle missing target values
            if y.isna().any():
                missing_targets = y.isna().sum()
                logger.warning(f"Removing {missing_targets} rows with missing target values")
                mask = ~y.isna()
                X, y = X[mask].reset_index(drop=True), y[mask].reset_index(drop=True)

            # Check if we still have enough data after cleaning
            if len(X) < 50:
                raise ValueError(f"After cleaning, only {len(X)} rows remain. Need at least 50 rows.")

            # Encode categorical variables
            categorical_cols = X.select_dtypes(["object", "category"]).columns
            if len(categorical_cols) > 0:
                logger.info(f"Encoding categorical columns: {list(categorical_cols)}")
                for col in categorical_cols:
                    le = LabelEncoder()
                    X[col] = le.fit_transform(X[col].astype(str))
                    self.label_encoders[col] = le

            # Handle target encoding
            if y.dtype == "object":
                logger.info("Encoding target variable")
                le_t = LabelEncoder()
                y = pd.Series(le_t.fit_transform(y), index=y.index)
                self.label_encoders["target"] = le_t

            # Check target distribution
            unique_targets = np.unique(y)
            self.is_binary = len(unique_targets) == 2
            logger.info(f"Detected {'binary' if self.is_binary else 'multi-class'} classification with {len(unique_targets)} classes")

            # Check for class imbalance
            target_counts = pd.Series(y).value_counts()
            min_class_count = target_counts.min()
            if min_class_count < 5:
                logger.warning(f"Warning: Minimum class has only {min_class_count} samples. This may cause training issues.")

            # Chronological split (prevents leakage)
            split_idx = int(len(X) * (1 - self.test_frac))
            
            # Ensure we have enough data in both splits
            if split_idx < 10 or (len(X) - split_idx) < 10:
                raise ValueError(f"Split results in insufficient data: train={split_idx}, test={len(X) - split_idx}")
            
            self.X_train = X.iloc[:split_idx].reset_index(drop=True)
            self.X_test = X.iloc[split_idx:].reset_index(drop=True)
            self.y_train = y.iloc[:split_idx].reset_index(drop=True)
            self.y_test = y.iloc[split_idx:].reset_index(drop=True)

            logger.info(f"Train set: {self.X_train.shape}, Test set: {self.X_test.shape}")
            
            # Validate feature consistency
            if self.X_train.shape[1] == 0:
                raise ValueError("No features remaining after preprocessing")
                
        except Exception as e:
            logger.error(f"Error in load_and_preprocess: {str(e)}")
            raise

    # ─── TRAIN ───────────────────────────────────────────────
    def train(self, best_params: dict):
        try:
            if self.X_train is None:
                raise RuntimeError("Data not loaded. Call load_and_preprocess() first.")
            
            logger.info("🚀 Starting model training...")
            
            params = best_params.copy()
            
            # Set appropriate objective and eval_metric
            if self.is_binary:
                params.update({
                    "objective": "binary:logistic",
                    "eval_metric": "logloss",  # Changed from aucpr for better stability
                })
            else:
                num_classes = len(np.unique(self.y_train))
                params.update({
                    "objective": "multi:softprob",
                    "eval_metric": "mlogloss",
                    "num_class": num_classes,
                })
            
            params.update({
                "random_state": self.random_state,
                "n_jobs": -1,
                "verbosity": 1,  # Enable some verbosity for debugging
                "tree_method": "hist",  # More efficient and stable
            })

            # Create validation split from training data
            val_frac = 0.2
            val_idx = int(len(self.X_train) * (1 - val_frac))
            
            # Ensure minimum validation size
            if (len(self.X_train) - val_idx) < 5:
                val_idx = len(self.X_train) - 5
                logger.warning("Adjusted validation split to ensure minimum validation size")
            
            X_tr = self.X_train.iloc[:val_idx]
            X_val = self.X_train.iloc[val_idx:]
            y_tr = self.y_train.iloc[:val_idx]
            y_val = self.y_train.iloc[val_idx:]

            logger.info(f"Training split: train={len(X_tr)}, val={len(X_val)}")

            # Initialize model
            self.model = xgb.XGBClassifier(**params)

            # Train model with error handling
            logger.info("Training XGBoost model...")
            
            # Check XGBoost version for callback compatibility
            xgb_version = xgb.__version__
            logger.info(f"XGBoost version: {xgb_version}")
            
            # Simple approach - just use early_stopping_rounds parameter
            try:
                self.model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    early_stopping_rounds=EARLY_STOP_ROUNDS,
                    verbose=False
                )
            except Exception as fit_error:
                logger.error(f"Error during model fitting: {fit_error}")
                # Try without early stopping as last resort
                logger.info("Trying without early stopping...")
                self.model.fit(X_tr, y_tr, verbose=False)

            # Get best iteration info
            if hasattr(self.model, 'best_iteration') and self.model.best_iteration is not None:
                best_iter = self.model.best_iteration
                logger.info(f"✓ Early stopping at iteration {best_iter}")
            else:
                best_iter = params.get('n_estimators', 100)
                logger.info(f"✓ Training completed with {best_iter} estimators")

        except Exception as e:
            logger.error(f"Error in training: {str(e)}")
            raise

    # ─── EVALUATE ────────────────────────────────────────────
    def evaluate(self, plots: bool = False):
        try:
            if self.model is None:
                raise RuntimeError("Model not trained. Call train() first!")

            logger.info("📊 Evaluating model...")

            # Make predictions
            y_hat_tr = self.model.predict(self.X_train)
            y_hat_te = self.model.predict(self.X_test)
            
            # Get prediction probabilities
            y_proba_tr = self.model.predict_proba(self.X_train)
            y_proba_te = self.model.predict_proba(self.X_test)

            if self.is_binary:
                y_proba_tr = y_proba_tr[:, 1]
                y_proba_te = y_proba_te[:, 1]

            # Calculate metrics
            avg = "binary" if self.is_binary else "weighted"
            
            metrics = {
                "accuracy_train": accuracy_score(self.y_train, y_hat_tr),
                "accuracy_test": accuracy_score(self.y_test, y_hat_te),
                "precision_test": precision_score(self.y_test, y_hat_te, average=avg, zero_division=0),
                "recall_test": recall_score(self.y_test, y_hat_te, average=avg, zero_division=0),
                "f1_test": f1_score(self.y_test, y_hat_te, average=avg, zero_division=0)
            }

            if self.is_binary:
                try:
                    metrics["roc_auc_test"] = roc_auc_score(self.y_test, y_proba_te)
                except ValueError as e:
                    logger.warning(f"Could not calculate ROC AUC: {e}")
                    metrics["roc_auc_test"] = None

            # Print results
            print("\n" + "="*50)
            print("MODEL EVALUATION RESULTS")
            print("="*50)
            for metric_name, value in metrics.items():
                if value is not None:
                    print(f"{metric_name:20}: {value:.4f}")
                else:
                    print(f"{metric_name:20}: N/A")
            print("="*50)

            return metrics

        except Exception as e:
            logger.error(f"Error in evaluation: {str(e)}")
            raise

    # ─── SAVE ────────────────────────────────────────────────
    def save(self, out_dir="xgb_final"):
        try:
            if self.model is None:
                raise RuntimeError("Model not trained. Call train() first!")
            
            os.makedirs(out_dir, exist_ok=True)
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Save model
            model_path = os.path.join(out_dir, f"xgb_model_{ts}.joblib")
            joblib.dump(self.model, model_path)
            logger.info(f"💾 Model saved to: {model_path}")

            # Save encoders
            if self.label_encoders:
                encoder_path = os.path.join(out_dir, f"encoders_{ts}.joblib")
                joblib.dump(self.label_encoders, encoder_path)
                logger.info(f"💾 Encoders saved to: {encoder_path}")

            # Save feature names
            if hasattr(self.X_train, "columns"):
                features_path = os.path.join(out_dir, f"features_{ts}.joblib")
                joblib.dump(list(self.X_train.columns), features_path)
                logger.info(f"💾 Feature names saved to: {features_path}")

            return model_path
            
        except Exception as e:
            logger.error(f"Error in saving: {str(e)}")
            raise


# ──────────────────────────────────────────────────────────────
#  MAIN
# ──────────────────────────────────────────────────────────────
def main():
    try:
        logger.info("🚀 Starting XGBoost training pipeline...")
        
        trainer = XGBoostFinalTrainer(
            data_path=DATA_PATH,
            target_column=TARGET_COLUMN,
            drop_columns=DROP_COLUMNS,
            test_frac=TEST_FRAC,
            random_state=RANDOM_SEED
        )
        
        # Load and preprocess data
        trainer.load_and_preprocess()
        
        # Train model
        trainer.train(BEST_PARAMS)
        
        # Evaluate model
        metrics = trainer.evaluate(plots=False)
        
        # Save model
        model_path = trainer.save()

        logger.info("🏁 Training pipeline completed successfully!")
        return trainer, metrics, model_path
        
    except Exception as e:
        logger.error(f"Training pipeline failed: {str(e)}")
        raise


if __name__ == "__main__":
    trainer, metrics, model_path = main()

INFO:__main__:🚀 Starting XGBoost training pipeline...
INFO:__main__:📥 Loading data from: C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv
INFO:__main__:Loaded data shape: (20718, 67)
INFO:__main__:Features shape: (20718, 66), Target shape: (20718,)
INFO:__main__:Target value counts:
target
1    10751
0     9967
Name: count, dtype: int64
INFO:__main__:Handling missing values...
INFO:__main__:Encoding categorical columns: ['timestamp']
INFO:__main__:Detected binary classification with 2 classes
INFO:__main__:Train set: (16574, 66), Test set: (4144, 66)
INFO:__main__:🚀 Starting model training...
INFO:__main__:Training split: train=13259, val=3315
INFO:__main__:Training XGBoost model...
INFO:__main__:XGBoost version: 3.0.2
ERROR:__main__:Error during model fitting: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
INFO:__main__:Trying without early stopping...
INFO:__main__:✓ T


MODEL EVALUATION RESULTS
accuracy_train      : 0.6066
accuracy_test       : 0.5198
precision_test      : 0.5381
recall_test         : 0.4423
f1_test             : 0.4855
roc_auc_test        : 0.5322


In [17]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def run_parameter_sweep():
    """
    Enhanced parameter sweep with better efficiency and analysis
    """
    
    # 🎯 PARAMETER COMBINATIONS TO TEST
    all_param_sets = [
        # TOP 5 BY PRECISION (from F0.5 optimization)
        {
            "n_estimators": 1307, "max_depth": 13, "learning_rate": 0.024718249520416128,
            "subsample": 0.9476077563455207, "colsample_bytree": 0.8927667771131523,
            "colsample_bylevel": 0.7500327051538141, "colsample_bynode": 0.9496606563368475,
            "reg_alpha": 0.002330119002013363, "reg_lambda": 0.0006033236716118722,
            "min_child_weight": 8, "gamma": 2.664811828490935, "max_bin": 481,
            "source": "precision_opt_trial18"
        },
        {
            "n_estimators": 534, "max_depth": 4, "learning_rate": 0.01898772399339208,
            "subsample": 0.6644885149016018, "colsample_bytree": 0.9718790609370292,
            "colsample_bylevel": 0.9232481518257668, "colsample_bynode": 0.8533615026041694,
            "reg_alpha": 0.6968596391373533, "reg_lambda": 0.1710207048797339,
            "min_child_weight": 2, "gamma": 4.462794992449889, "max_bin": 394,
            "source": "precision_opt_trial9"
        },
        {
            "n_estimators": 966, "max_depth": 13, "learning_rate": 0.01254057843022616,
            "subsample": 0.8056937753654446, "colsample_bytree": 0.836965827544817,
            "colsample_bylevel": 0.6185801650879991, "colsample_bynode": 0.8430179407605753,
            "reg_alpha": 3.425445902633376e-07, "reg_lambda": 3.850031979199519e-08,
            "min_child_weight": 10, "gamma": 4.828160165372797, "max_bin": 463,
            "source": "precision_opt_trial2"
        },
        {
            "n_estimators": 1297, "max_depth": 14, "learning_rate": 0.032677079513305216,
            "subsample": 0.920158613666429, "colsample_bytree": 0.8176021736804028,
            "colsample_bylevel": 0.635663367103597, "colsample_bynode": 0.8074879828556969,
            "reg_alpha": 0.00422918163231331, "reg_lambda": 2.6434085113630614e-07,
            "min_child_weight": 7, "gamma": 4.315961451395856, "max_bin": 464,
            "source": "precision_opt_trial25"
        },
        {
            "n_estimators": 1869, "max_depth": 11, "learning_rate": 0.029785466048449057,
            "subsample": 0.9600239834936011, "colsample_bytree": 0.7984617451987015,
            "colsample_bylevel": 0.7596781750650533, "colsample_bynode": 0.9578920226477285,
            "reg_alpha": 1.8469408002336966e-07, "reg_lambda": 2.687788315138241e-06,
            "min_child_weight": 10, "gamma": 4.55767033074189, "max_bin": 484,
            "source": "precision_opt_trial22"
        },
        
        # TOP 5 BY F1 SCORE (from F0.5 optimization)
        {
            "n_estimators": 105, "max_depth": 8, "learning_rate": 0.005040041472632554,
            "subsample": 0.7315045755029569, "colsample_bytree": 0.6978876037786749,
            "colsample_bylevel": 0.8270460042189236, "colsample_bynode": 0.6107312168264377,
            "reg_alpha": 2.2521102114594034, "reg_lambda": 0.00038525640227419795,
            "min_child_weight": 6, "gamma": 3.194247833014993, "max_bin": 391,
            "source": "f1_opt_trial76"
        },
        {
            "n_estimators": 107, "max_depth": 7, "learning_rate": 0.007331979140441173,
            "subsample": 0.6772174715021827, "colsample_bytree": 0.8238805270933549,
            "colsample_bylevel": 0.7730466620963483, "colsample_bynode": 0.6682429822080043,
            "reg_alpha": 5.409756188443174, "reg_lambda": 0.0015524839266472987,
            "min_child_weight": 6, "gamma": 3.7995867106505936, "max_bin": 365,
            "source": "f1_opt_trial63"
        },
        {
            "n_estimators": 152, "max_depth": 10, "learning_rate": 0.005240578329731185,
            "subsample": 0.7651730607230356, "colsample_bytree": 0.6318385953435955,
            "colsample_bylevel": 0.916547265646687, "colsample_bynode": 0.6225529059741628,
            "reg_alpha": 6.128116326056999, "reg_lambda": 0.019433498280224788,
            "min_child_weight": 5, "gamma": 2.424062851363187, "max_bin": 410,
            "source": "f1_opt_trial99"
        },
        {
            "n_estimators": 103, "max_depth": 9, "learning_rate": 0.005217000368658448,
            "subsample": 0.7308153424745538, "colsample_bytree": 0.6964248488818299,
            "colsample_bylevel": 0.8778333045131583, "colsample_bynode": 0.6116428797860312,
            "reg_alpha": 0.7200783783474446, "reg_lambda": 0.0002441557634829753,
            "min_child_weight": 6, "gamma": 3.2496190417438267, "max_bin": 408,
            "source": "f1_opt_trial77"
        },
        {
            "n_estimators": 153, "max_depth": 10, "learning_rate": 0.0050016463516870035,
            "subsample": 0.7648437723083729, "colsample_bytree": 0.6550043886733103,
            "colsample_bylevel": 0.8978287244153174, "colsample_bynode": 0.6006227499667893,
            "reg_alpha": 2.5277019263753244, "reg_lambda": 0.009024542068296396,
            "min_child_weight": 5, "gamma": 2.6021269907013367, "max_bin": 430,
            "source": "f1_opt_trial98"
        }
    ]
    
    # 📊 RESULTS STORAGE
    results = []
    failed_runs = []
    start_time = time.time()
    
    # 🔄 LOAD DATA ONCE (efficiency improvement)
    logger.info("🔄 Loading and preprocessing data once for all runs...")
    try:
        base_trainer = XGBoostFinalTrainer(
            data_path=DATA_PATH,
            target_column=TARGET_COLUMN,
            drop_columns=DROP_COLUMNS,
            test_frac=TEST_FRAC,
            random_state=RANDOM_SEED
        )
        base_trainer.load_and_preprocess()
        logger.info("✅ Data loaded successfully!")
    except Exception as e:
        logger.error(f"❌ Failed to load data: {e}")
        return None, None
    
    # 🏃‍♂️ RUN PARAMETER SWEEP
    total_runs = len(all_param_sets)
    logger.info(f"🚀 Starting parameter sweep with {total_runs} configurations...")
    
    for i, param_set in enumerate(all_param_sets, 1):
        run_start = time.time()
        params = param_set.copy()
        source = params.pop("source", "unknown")
        
        logger.info(f"\n⚙️  Run {i}/{total_runs} ({source})")
        logger.info(f"    Params: {params}")
        
        try:
            # Create new trainer instance with same preprocessed data
            trainer = XGBoostFinalTrainer(
                data_path=DATA_PATH,
                target_column=TARGET_COLUMN,
                drop_columns=DROP_COLUMNS,
                test_frac=TEST_FRAC,
                random_state=RANDOM_SEED
            )
            
            # Copy preprocessed data instead of reloading
            trainer.X_train = base_trainer.X_train.copy()
            trainer.X_test = base_trainer.X_test.copy()
            trainer.y_train = base_trainer.y_train.copy()
            trainer.y_test = base_trainer.y_test.copy()
            trainer.is_binary = base_trainer.is_binary
            trainer.label_encoders = base_trainer.label_encoders.copy()
            
            # Train with current parameters
            trainer.train(params)
            metrics = trainer.evaluate(plots=False)
            
            run_time = time.time() - run_start
            
            # Store results
            result = {
                "run_id": i,
                "source": source,
                "training_time_sec": round(run_time, 2),
                
                # Parameters
                "n_estimators": params.get("n_estimators"),
                "max_depth": params.get("max_depth"),
                "learning_rate": params.get("learning_rate"),
                "reg_alpha": params.get("reg_alpha"),
                
                # Metrics
                "accuracy_train": metrics.get("accuracy_train", 0),
                "accuracy_test": metrics.get("accuracy_test", 0),
                "precision_test": metrics.get("precision_test", 0),
                "recall_test": metrics.get("recall_test", 0),
                "f1_test": metrics.get("f1_test", 0),
                "roc_auc_test": metrics.get("roc_auc_test", 0),
                
                # Overfitting indicator
                "overfitting": metrics.get("accuracy_train", 0) - metrics.get("accuracy_test", 0)
            }
            
            results.append(result)
            logger.info(f"✅ Run {i} completed in {run_time:.1f}s - F1: {result['f1_test']:.4f}")
            
        except Exception as e:
            run_time = time.time() - run_start
            logger.error(f"❌ Run {i} failed after {run_time:.1f}s: {str(e)[:100]}...")
            failed_runs.append({
                "run_id": i,
                "source": source,
                "params": params,
                "error": str(e)[:200]
            })
    
    # 📈 ANALYSIS AND RESULTS
    total_time = time.time() - start_time
    logger.info(f"\n🏁 Parameter sweep completed in {total_time:.1f} seconds")
    logger.info(f"✅ Successful runs: {len(results)}/{total_runs}")
    logger.info(f"❌ Failed runs: {len(failed_runs)}/{total_runs}")
    
    if not results:
        logger.error("No successful runs - cannot proceed with analysis")
        return None, pd.DataFrame(failed_runs) if failed_runs else None
    
    # Convert to DataFrame and analyze
    df_results = pd.DataFrame(results)
    
    # 🏆 FIND BEST MODELS
    best_f1 = df_results.loc[df_results['f1_test'].idxmax()]
    best_precision = df_results.loc[df_results['precision_test'].idxmax()]
    best_recall = df_results.loc[df_results['recall_test'].idxmax()]
    best_balanced = df_results.loc[df_results['accuracy_test'].idxmax()]
    
    # Sort by F1 score (or change to your preferred metric)
    df_results = df_results.sort_values(by="f1_test", ascending=False).reset_index(drop=True)
    
    # 📊 SUMMARY REPORT
    print("\n" + "="*80)
    print("🏆 PARAMETER SWEEP RESULTS")
    print("="*80)
    
    print(f"\n📈 BEST MODELS:")
    print(f"🥇 Best F1 Score:     {best_f1['f1_test']:.4f} (Run {best_f1['run_id']})")
    print(f"🎯 Best Precision:    {best_precision['precision_test']:.4f} (Run {best_precision['run_id']})")
    print(f"🔍 Best Recall:       {best_recall['recall_test']:.4f} (Run {best_recall['run_id']})")
    print(f"⚖️  Best Accuracy:     {best_balanced['accuracy_test']:.4f} (Run {best_balanced['run_id']})")
    
    print(f"\n📊 TOP 5 MODELS BY F1 SCORE:")
    top_5 = df_results.head(5)[['run_id', 'source', 'f1_test', 'precision_test', 'recall_test', 'accuracy_test', 'training_time_sec']]
    print(top_5.to_string(index=False, float_format='%.4f'))
    
    if len(failed_runs) > 0:
        print(f"\n❌ FAILED RUNS:")
        df_failed = pd.DataFrame(failed_runs)
        print(df_failed[['run_id', 'source', 'error']].to_string(index=False))
    
    # 📋 DETAILED RESULTS
    print(f"\n📋 ALL RESULTS (sorted by F1 score):")
    display_cols = ['run_id', 'source', 'f1_test', 'precision_test', 'recall_test', 
                   'accuracy_test', 'overfitting', 'training_time_sec']
    print(df_results[display_cols].to_string(index=False, float_format='%.4f'))
    
    print("="*80)
    
    return df_results, df_failed if failed_runs else None


def get_best_params(df_results, metric='f1_test'):
    """
    Extract the best parameter set based on specified metric
    """
    if df_results is None or df_results.empty:
        return None
    
    best_row = df_results.loc[df_results[metric].idxmax()]
    
    best_params = {
        "n_estimators": int(best_row["n_estimators"]),
        "max_depth": int(best_row["max_depth"]),
        "learning_rate": float(best_row["learning_rate"]),
        "reg_alpha": float(best_row["reg_alpha"])
    }
    
    print(f"\n🎯 BEST PARAMETERS (by {metric}):")
    print(f"Score: {best_row[metric]:.4f}")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    
    return best_params


def save_results(df_results, df_failed=None, output_dir="parameter_sweep_results"):
    """
    Save results to CSV files with timestamp
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if df_results is not None:
        results_file = os.path.join(output_dir, f"parameter_sweep_results_{timestamp}.csv")
        df_results.to_csv(results_file, index=False)
        logger.info(f"💾 Results saved to: {results_file}")
    
    if df_failed is not None and not df_failed.empty:
        failed_file = os.path.join(output_dir, f"parameter_sweep_failed_{timestamp}.csv")
        df_failed.to_csv(failed_file, index=False)
        logger.info(f"💾 Failed runs saved to: {failed_file}")


# 🚀 UPDATED MAIN FUNCTION
def main():
    """
    Main function to run parameter sweep
    """
    try:
        # Run parameter sweep
        df_results, df_failed = run_parameter_sweep()
        
        if df_results is not None:
            # Get best parameters
            best_params = get_best_params(df_results, metric='f1_test')
            
            # Save results
            save_results(df_results, df_failed)
            
            return df_results, df_failed, best_params
        else:
            logger.error("Parameter sweep failed completely")
            return None, df_failed, None
            
    except Exception as e:
        logger.error(f"Main function failed: {e}")
        raise


if __name__ == "__main__":
    df_results, df_failed, best_params = main()

INFO:__main__:🔄 Loading and preprocessing data once for all runs...
INFO:__main__:📥 Loading data from: C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv
INFO:__main__:Loaded data shape: (20718, 67)
INFO:__main__:Features shape: (20718, 66), Target shape: (20718,)
INFO:__main__:Target value counts:
target
1    10751
0     9967
Name: count, dtype: int64
INFO:__main__:Handling missing values...
INFO:__main__:Encoding categorical columns: ['timestamp']
INFO:__main__:Detected binary classification with 2 classes
INFO:__main__:Train set: (16574, 66), Test set: (4144, 66)
INFO:__main__:✅ Data loaded successfully!
INFO:__main__:🚀 Starting parameter sweep with 10 configurations...
INFO:__main__:
⚙️  Run 1/10 (precision_opt_trial18)
INFO:__main__:    Params: {'n_estimators': 1307, 'max_depth': 13, 'learning_rate': 0.024718249520416128, 'subsample': 0.9476077563455207, 'colsample_bytree': 0.8927667771131523, 'cols


MODEL EVALUATION RESULTS
accuracy_train      : 0.8313
accuracy_test       : 0.5133
precision_test      : 0.5315
recall_test         : 0.4216
f1_test             : 0.4702
roc_auc_test        : 0.5300


INFO:__main__:✓ Training completed with 534 estimators
INFO:__main__:📊 Evaluating model...
INFO:__main__:✅ Run 2 completed in 0.7s - F1: 0.4748
INFO:__main__:
⚙️  Run 3/10 (precision_opt_trial2)
INFO:__main__:    Params: {'n_estimators': 966, 'max_depth': 13, 'learning_rate': 0.01254057843022616, 'subsample': 0.8056937753654446, 'colsample_bytree': 0.836965827544817, 'colsample_bylevel': 0.6185801650879991, 'colsample_bynode': 0.8430179407605753, 'reg_alpha': 3.425445902633376e-07, 'reg_lambda': 3.850031979199519e-08, 'min_child_weight': 10, 'gamma': 4.828160165372797, 'max_bin': 463}
INFO:__main__:🚀 Starting model training...
INFO:__main__:Training split: train=13259, val=3315
INFO:__main__:Training XGBoost model...
INFO:__main__:XGBoost version: 3.0.2
ERROR:__main__:Error during model fitting: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
INFO:__main__:Trying without early stopping...



MODEL EVALUATION RESULTS
accuracy_train      : 0.6321
accuracy_test       : 0.5270
precision_test      : 0.5507
recall_test         : 0.4173
f1_test             : 0.4748
roc_auc_test        : 0.5344


INFO:__main__:✓ Training completed with 966 estimators
INFO:__main__:📊 Evaluating model...
INFO:__main__:✅ Run 3 completed in 1.5s - F1: 0.4982
INFO:__main__:
⚙️  Run 4/10 (precision_opt_trial25)
INFO:__main__:    Params: {'n_estimators': 1297, 'max_depth': 14, 'learning_rate': 0.032677079513305216, 'subsample': 0.920158613666429, 'colsample_bytree': 0.8176021736804028, 'colsample_bylevel': 0.635663367103597, 'colsample_bynode': 0.8074879828556969, 'reg_alpha': 0.00422918163231331, 'reg_lambda': 2.6434085113630614e-07, 'min_child_weight': 7, 'gamma': 4.315961451395856, 'max_bin': 464}
INFO:__main__:🚀 Starting model training...
INFO:__main__:Training split: train=13259, val=3315
INFO:__main__:Training XGBoost model...
INFO:__main__:XGBoost version: 3.0.2
ERROR:__main__:Error during model fitting: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
INFO:__main__:Trying without early stopping...


...


MODEL EVALUATION RESULTS
accuracy_train      : 0.5947
accuracy_test       : 0.5244
precision_test      : 0.5220
recall_test         : 0.8479
f1_test             : 0.6462
roc_auc_test        : 0.5524


INFO:__main__:✓ Training completed with 103 estimators
INFO:__main__:📊 Evaluating model...
INFO:__main__:✅ Run 9 completed in 0.7s - F1: 0.6241
INFO:__main__:
⚙️  Run 10/10 (f1_opt_trial98)
INFO:__main__:    Params: {'n_estimators': 153, 'max_depth': 10, 'learning_rate': 0.0050016463516870035, 'subsample': 0.7648437723083729, 'colsample_bytree': 0.6550043886733103, 'colsample_bylevel': 0.8978287244153174, 'colsample_bynode': 0.6006227499667893, 'reg_alpha': 2.5277019263753244, 'reg_lambda': 0.009024542068296396, 'min_child_weight': 5, 'gamma': 2.6021269907013367, 'max_bin': 430}
INFO:__main__:🚀 Starting model training...
INFO:__main__:Training split: train=13259, val=3315
INFO:__main__:Training XGBoost model...
INFO:__main__:XGBoost version: 3.0.2
ERROR:__main__:Error during model fitting: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
INFO:__main__:Trying without early stopping...



MODEL EVALUATION RESULTS
accuracy_train      : 0.6274
accuracy_test       : 0.5282
precision_test      : 0.5273
recall_test         : 0.7645
f1_test             : 0.6241
roc_auc_test        : 0.5428


INFO:__main__:✓ Training completed with 153 estimators
INFO:__main__:📊 Evaluating model...
INFO:__main__:✅ Run 10 completed in 1.0s - F1: 0.6190
INFO:__main__:
🏁 Parameter sweep completed in 10.6 seconds
INFO:__main__:✅ Successful runs: 10/10
INFO:__main__:❌ Failed runs: 0/10
INFO:__main__:💾 Results saved to: parameter_sweep_results\parameter_sweep_results_20250612_171847.csv



MODEL EVALUATION RESULTS
accuracy_train      : 0.6575
accuracy_test       : 0.5316
precision_test      : 0.5306
recall_test         : 0.7428
f1_test             : 0.6190
roc_auc_test        : 0.5437

🏆 PARAMETER SWEEP RESULTS

📈 BEST MODELS:
🥇 Best F1 Score:     0.6481 (Run 7)
🎯 Best Precision:    0.5507 (Run 2)
🔍 Best Recall:       0.8479 (Run 8)
⚖️  Best Accuracy:     0.5343 (Run 7)

📊 TOP 5 MODELS BY F1 SCORE:
 run_id         source  f1_test  precision_test  recall_test  accuracy_test  training_time_sec
      7 f1_opt_trial63   0.6481          0.5287       0.8370         0.5343             0.2900
      8 f1_opt_trial99   0.6462          0.5220       0.8479         0.5244             0.6500
      6 f1_opt_trial76   0.6430          0.5207       0.8403         0.5220             0.4500
      9 f1_opt_trial77   0.6241          0.5273       0.7645         0.5282             0.6600
     10 f1_opt_trial98   0.6190          0.5306       0.7428         0.5316             1.0400

📋 ALL RESUL

In [ ]:
# =============================================================
#  Trial 9 (Best Precision) Model Trainer & Predictor
# =============================================================
import os, warnings, joblib
from datetime import datetime
import logging

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report)
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ──────────────────────────────────────────────────────────────
#  CONFIG - TRIAL 9 BEST PRECISION PARAMETERS
# ──────────────────────────────────────────────────────────────
DATA_PATH = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"
TARGET_COLUMN = "target"
DROP_COLUMNS = ["timestamp"]  # We'll keep timestamp for predictions
TEST_FRAC = 0.20
RANDOM_SEED = 42

# TRIAL 9 BEST PRECISION PARAMETERS
TRIAL9_PARAMS = {
    "n_estimators": 534,
    "max_depth": 4,
    "learning_rate": 0.01898772399339208,
    "subsample": 0.6644885149016018,
    "colsample_bytree": 0.9718790609370292,
    "colsample_bylevel": 0.9232481518257668,
    "colsample_bynode": 0.8533615026041694,
    "reg_alpha": 0.6968596391373533,
    "reg_lambda": 0.1710207048797339,
    "min_child_weight": 2,
    "gamma": 4.462794992449889,
    "max_bin": 394,
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "random_state": RANDOM_SEED,
    "verbosity": 1,
    "tree_method": "hist",
}


class Trial9Trainer:
    def __init__(self):
        self.model = None
        self.label_encoders = {}
        self.feature_columns = None
        self.X_train = self.X_test = self.y_train = self.y_test = None
        self.test_timestamps = None
        
    def load_and_preprocess(self):
        """Load and preprocess data, keeping timestamps for predictions"""
        logger.info("📥 Loading data for Trial 9 training...")
        
        if not os.path.exists(DATA_PATH):
            raise FileNotFoundError(f"Data file not found: {DATA_PATH}")
        
        df = pd.read_csv(DATA_PATH)
        logger.info(f"Loaded data shape: {df.shape}")
        
        if TARGET_COLUMN not in df.columns:
            raise KeyError(f"Target column '{TARGET_COLUMN}' not found")

        # Keep timestamps for later use
        timestamps = df['timestamp'].copy()
        
        # Separate target and features
        y = df[TARGET_COLUMN].copy()
        X = df.drop(columns=[TARGET_COLUMN])  # Keep timestamp initially
        
        logger.info(f"Target distribution:\n{y.value_counts()}")

        # Handle missing values in features (excluding timestamp)
        feature_cols = [col for col in X.columns if col != 'timestamp']
        for col in feature_cols:
            if X[col].dtype in ("object", "category"):
                mode_val = X[col].mode()
                fill_val = mode_val.iloc[0] if len(mode_val) > 0 else "unknown"
                X[col] = X[col].fillna(fill_val)
            else:
                X[col] = X[col].fillna(X[col].median())

        # Handle missing targets
        if y.isna().any():
            mask = ~y.isna()
            X, y, timestamps = X[mask].reset_index(drop=True), y[mask].reset_index(drop=True), timestamps[mask].reset_index(drop=True)

        # Encode categorical variables (timestamp)
        le_timestamp = LabelEncoder()
        X['timestamp'] = le_timestamp.fit_transform(X['timestamp'].astype(str))
        self.label_encoders['timestamp'] = le_timestamp

        # Handle target encoding if needed
        if y.dtype == "object":
            le_target = LabelEncoder()
            y = pd.Series(le_target.fit_transform(y), index=y.index)
            self.label_encoders["target"] = le_target

        logger.info(f"Detected binary classification with classes: {np.unique(y)}")

        # CHRONOLOGICAL SPLIT (keeping timestamps)
        split_idx = int(len(X) * (1 - TEST_FRAC))
        
        self.X_train = X.iloc[:split_idx].reset_index(drop=True)
        self.X_test = X.iloc[split_idx:].reset_index(drop=True)
        self.y_train = y.iloc[:split_idx].reset_index(drop=True)
        self.y_test = y.iloc[split_idx:].reset_index(drop=True)
        
        # Store test timestamps for predictions (before dropping timestamp column)
        self.test_timestamps = timestamps.iloc[split_idx:].reset_index(drop=True)
        
        # Now drop timestamp from features for training
        self.X_train = self.X_train.drop(columns=['timestamp'])
        self.X_test = self.X_test.drop(columns=['timestamp'])
        
        # Store feature column names
        self.feature_columns = list(self.X_train.columns)
        
        logger.info(f"Train set: {self.X_train.shape}, Test set: {self.X_test.shape}")
        logger.info(f"Features: {len(self.feature_columns)}")

    def train(self):
        """Train the Trial 9 model"""
        logger.info("🚀 Training Trial 9 (Best Precision) model...")
        
        self.model = xgb.XGBClassifier(**TRIAL9_PARAMS)
        
        # Train model
        self.model.fit(self.X_train, self.y_train, verbose=False)
        
        logger.info("✓ Training completed!")

    def evaluate(self):
        """Evaluate the trained model"""
        if self.model is None:
            raise RuntimeError("Model not trained!")

        logger.info("📊 Evaluating Trial 9 model...")

        # Predictions
        y_train_pred = self.model.predict(self.X_train)
        y_test_pred = self.model.predict(self.X_test)
        y_test_proba = self.model.predict_proba(self.X_test)
        
        # Calculate metrics
        train_acc = accuracy_score(self.y_train, y_train_pred)
        test_acc = accuracy_score(self.y_test, y_test_pred)
        test_precision = precision_score(self.y_test, y_test_pred, zero_division=0)
        test_recall = recall_score(self.y_test, y_test_pred, zero_division=0)
        test_f1 = f1_score(self.y_test, y_test_pred, zero_division=0)
        test_auc = roc_auc_score(self.y_test, y_test_proba[:, 1])
        
        print("\n" + "="*60)
        print("🎯 TRIAL 9 (BEST PRECISION) MODEL RESULTS")
        print("="*60)
        print(f"Training Accuracy:   {train_acc:.4f}")
        print(f"Test Accuracy:       {test_acc:.4f}")
        print(f"Test Precision:      {test_precision:.4f} 🎯")
        print(f"Test Recall:         {test_recall:.4f}")
        print(f"Test F1:             {test_f1:.4f}")
        print(f"Test AUC:            {test_auc:.4f}")
        print(f"Overfitting Gap:     {train_acc - test_acc:.4f}")
        print("="*60)
        
        return {
            "train_accuracy": train_acc,
            "test_accuracy": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f1": test_f1,
            "test_auc": test_auc
        }

    def generate_predictions_csv(self, output_file="trial9_predictions.csv"):
        """Generate predictions CSV in the requested format"""
        if self.model is None:
            raise RuntimeError("Model not trained!")

        logger.info("📊 Generating predictions CSV...")
        
        # Get probabilities for test set
        y_test_proba = self.model.predict_proba(self.X_test)
        y_test_pred = self.model.predict(self.X_test)
        
        # Create predictions DataFrame
        predictions_df = pd.DataFrame({
            'timestamp': self.test_timestamps,
            'prob_down': y_test_proba[:, 0],  # Probability of class 0
            'prob_up': y_test_proba[:, 1],    # Probability of class 1
            'winning_prob': np.maximum(y_test_proba[:, 0], y_test_proba[:, 1]),  # Higher probability
            'prediction': y_test_pred,
            'actual': self.y_test
        })
        
        # Round probabilities to 6 decimal places to match your format
        predictions_df['prob_down'] = predictions_df['prob_down'].round(6)
        predictions_df['prob_up'] = predictions_df['prob_up'].round(6)
        predictions_df['winning_prob'] = predictions_df['winning_prob'].round(6)
        
        # Save to CSV
        predictions_df.to_csv(output_file, index=False)
        
        logger.info(f"💾 Predictions saved to: {output_file}")
        logger.info(f"📊 Total predictions: {len(predictions_df)}")
        
        # Show sample of predictions
        print("\n🔍 SAMPLE PREDICTIONS:")
        print(predictions_df.head(10).to_string(index=False))
        
        # Calculate accuracy on predictions
        accuracy = accuracy_score(predictions_df['actual'], predictions_df['prediction'])
        precision = precision_score(predictions_df['actual'], predictions_df['prediction'], zero_division=0)
        
        print(f"\n📈 PREDICTION SUMMARY:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Correct predictions: {(predictions_df['prediction'] == predictions_df['actual']).sum()}/{len(predictions_df)}")
        
        return predictions_df

    def save_model(self, output_dir="trial9_model"):
        """Save the trained model"""
        if self.model is None:
            raise RuntimeError("Model not trained!")
        
        os.makedirs(output_dir, exist_ok=True)
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save model
        model_path = os.path.join(output_dir, f"trial9_best_precision_{ts}.joblib")
        joblib.dump(self.model, model_path)
        
        # Save encoders and feature names
        if self.label_encoders:
            joblib.dump(self.label_encoders, os.path.join(output_dir, f"trial9_encoders_{ts}.joblib"))
        
        joblib.dump(self.feature_columns, os.path.join(output_dir, f"trial9_features_{ts}.joblib"))
        
        logger.info(f"💾 Trial 9 model saved to: {model_path}")
        return model_path


def main():
    """Main function to train Trial 9 model and generate predictions"""
    logger.info("🚀 Starting Trial 9 (Best Precision) training pipeline...")
    
    # Initialize trainer
    trainer = Trial9Trainer()
    
    try:
        # Load and preprocess data
        trainer.load_and_preprocess()
        
        # Train model with Trial 9 parameters
        trainer.train()
        
        # Evaluate model
        metrics = trainer.evaluate()
        
        # Generate predictions CSV
        predictions_df = trainer.generate_predictions_csv("trial9_predictions.csv")
        
        # Save model
        model_path = trainer.save_model()
        
        logger.info("🏁 Trial 9 training and prediction generation completed!")
        
        return trainer, metrics, predictions_df
        
    except Exception as e:
        logger.error(f"Pipeline failed: {str(e)}")
        raise


if __name__ == "__main__":
    trainer, metrics, predictions_df = main()

INFO:__main__:🚀 Starting Trial 9 (Best Precision) training pipeline...
INFO:__main__:📥 Loading data for Trial 9 training...
INFO:__main__:Loaded data shape: (20718, 67)
INFO:__main__:Target distribution:
target
1    10751
0     9967
Name: count, dtype: int64
INFO:__main__:Detected binary classification with classes: [0 1]
INFO:__main__:Train set: (16574, 65), Test set: (4144, 65)
INFO:__main__:Features: 65
INFO:__main__:🚀 Training Trial 9 (Best Precision) model...
INFO:__main__:✓ Training completed!
INFO:__main__:📊 Evaluating Trial 9 model...
INFO:__main__:📊 Generating predictions CSV...
INFO:__main__:💾 Predictions saved to: trial9_predictions.csv
INFO:__main__:📊 Total predictions: 4144
INFO:__main__:💾 Trial 9 model saved to: trial9_model\trial9_best_precision_20250612_172752.joblib
INFO:__main__:🏁 Trial 9 training and prediction generation completed!



🎯 TRIAL 9 (BEST PRECISION) MODEL RESULTS
Training Accuracy:   0.6476
Test Accuracy:       0.5263
Test Precision:      0.5567 🎯
Test Recall:         0.3702
Test F1:             0.4447
Test AUC:            0.5396
Overfitting Gap:     0.1213

🔍 SAMPLE PREDICTIONS:
          timestamp  prob_down  prob_up  winning_prob  prediction  actual
2023-05-07 12:00:00   0.523782 0.476218      0.523782           0       0
2023-05-07 16:00:00   0.591859 0.408141      0.591859           0       0
2023-05-07 20:00:00   0.520758 0.479242      0.520758           0       0
2023-05-08 00:00:00   0.447865 0.552135      0.552135           1       0
2023-05-08 04:00:00   0.375828 0.624172      0.624172           1       0
2023-05-08 08:00:00   0.428110 0.571890      0.571890           1       1
2023-05-08 12:00:00   0.450084 0.549916      0.549916           1       1
2023-05-08 16:00:00   0.522540 0.477460      0.522540           0       0
2023-05-08 20:00:00   0.450983 0.549017      0.549017           1      

In [1]:
# =============================================================
#  XGBoost Model Trainer & Predictor (Fixed Timestamps)
# =============================================================
import os, warnings, joblib
from datetime import datetime
import logging

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report)
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ──────────────────────────────────────────────────────────────
#  CONFIG - XGBOOST PARAMETERS
# ──────────────────────────────────────────────────────────────
DATA_PATH = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"
OUTPUT_PATH = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Final_runs_csv"
TARGET_COLUMN = "target"
DROP_COLUMNS = ["timestamp"]  # We'll keep timestamp for predictions
TEST_FRAC = 0.20
RANDOM_SEED = 42

# You can adjust these XGBoost parameters as needed
XGBOOST_PARAMS = {
    "n_estimators": 500,
    "max_depth": 6,
    "learning_rate": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "min_child_weight": 1,
    "gamma": 0.1,
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "random_state": RANDOM_SEED,
    "verbosity": 1,
    "tree_method": "hist",
}


class XGBoostTrainer:
    def __init__(self):
        self.model = None
        self.label_encoders = {}
        self.feature_columns = None
        self.X_train = self.X_test = self.y_train = self.y_test = None
        self.test_timestamps = None
        
    def load_and_preprocess(self):
        """Load and preprocess data, keeping timestamps for predictions"""
        logger.info("📥 Loading data for XGBoost training...")
        
        if not os.path.exists(DATA_PATH):
            raise FileNotFoundError(f"Data file not found: {DATA_PATH}")
        
        df = pd.read_csv(DATA_PATH)
        logger.info(f"Loaded data shape: {df.shape}")
        
        if TARGET_COLUMN not in df.columns:
            raise KeyError(f"Target column '{TARGET_COLUMN}' not found")

        # Parse timestamps properly to ensure consistent format
        logger.info("🕐 Parsing timestamps...")
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Keep timestamps for later use
        timestamps = df['timestamp'].copy()
        
        # Separate target and features
        y = df[TARGET_COLUMN].copy()
        X = df.drop(columns=[TARGET_COLUMN])  # Keep timestamp initially
        
        logger.info(f"Target distribution:\n{y.value_counts()}")

        # Handle missing values in features (excluding timestamp)
        feature_cols = [col for col in X.columns if col != 'timestamp']
        for col in feature_cols:
            if X[col].dtype in ("object", "category"):
                mode_val = X[col].mode()
                fill_val = mode_val.iloc[0] if len(mode_val) > 0 else "unknown"
                X[col] = X[col].fillna(fill_val)
            else:
                X[col] = X[col].fillna(X[col].median())

        # Handle missing targets
        if y.isna().any():
            mask = ~y.isna()
            X, y, timestamps = X[mask].reset_index(drop=True), y[mask].reset_index(drop=True), timestamps[mask].reset_index(drop=True)

        # Don't encode timestamp - we'll drop it for training but keep original
        # This avoids any timestamp format issues
        
        # Handle target encoding if needed
        if y.dtype == "object":
            le_target = LabelEncoder()
            y = pd.Series(le_target.fit_transform(y), index=y.index)
            self.label_encoders["target"] = le_target

        logger.info(f"Detected binary classification with classes: {np.unique(y)}")

        # CHRONOLOGICAL SPLIT (keeping timestamps)
        split_idx = int(len(X) * (1 - TEST_FRAC))
        
        self.X_train = X.iloc[:split_idx].reset_index(drop=True)
        self.X_test = X.iloc[split_idx:].reset_index(drop=True)
        self.y_train = y.iloc[:split_idx].reset_index(drop=True)
        self.y_test = y.iloc[split_idx:].reset_index(drop=True)
        
        # Store test timestamps for predictions (before dropping timestamp column)
        self.test_timestamps = timestamps.iloc[split_idx:].reset_index(drop=True)
        
        # Now drop timestamp from features for training
        self.X_train = self.X_train.drop(columns=['timestamp'])
        self.X_test = self.X_test.drop(columns=['timestamp'])
        
        # Store feature column names
        self.feature_columns = list(self.X_train.columns)
        
        logger.info(f"Train set: {self.X_train.shape}, Test set: {self.X_test.shape}")
        logger.info(f"Features: {len(self.feature_columns)}")
        logger.info(f"Test date range: {self.test_timestamps.min()} to {self.test_timestamps.max()}")

    def train(self):
        """Train the XGBoost model"""
        logger.info("🚀 Training XGBoost model...")
        
        self.model = xgb.XGBClassifier(**XGBOOST_PARAMS)
        
        # Train model
        self.model.fit(self.X_train, self.y_train, verbose=False)
        
        logger.info("✓ Training completed!")

    def evaluate(self):
        """Evaluate the trained model"""
        if self.model is None:
            raise RuntimeError("Model not trained!")

        logger.info("📊 Evaluating XGBoost model...")

        # Predictions
        y_train_pred = self.model.predict(self.X_train)
        y_test_pred = self.model.predict(self.X_test)
        y_test_proba = self.model.predict_proba(self.X_test)
        
        # Calculate metrics
        train_acc = accuracy_score(self.y_train, y_train_pred)
        test_acc = accuracy_score(self.y_test, y_test_pred)
        test_precision = precision_score(self.y_test, y_test_pred, zero_division=0)
        test_recall = recall_score(self.y_test, y_test_pred, zero_division=0)
        test_f1 = f1_score(self.y_test, y_test_pred, zero_division=0)
        test_auc = roc_auc_score(self.y_test, y_test_proba[:, 1])
        
        print("\n" + "="*60)
        print("🎯 XGBOOST MODEL RESULTS")
        print("="*60)
        print(f"Training Accuracy:   {train_acc:.4f}")
        print(f"Test Accuracy:       {test_acc:.4f}")
        print(f"Test Precision:      {test_precision:.4f}")
        print(f"Test Recall:         {test_recall:.4f}")
        print(f"Test F1:             {test_f1:.4f}")
        print(f"Test AUC:            {test_auc:.4f}")
        print(f"Overfitting Gap:     {train_acc - test_acc:.4f}")
        print("="*60)
        
        # Show classification report
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_test_pred))
        
        return {
            "train_accuracy": train_acc,
            "test_accuracy": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f1": test_f1,
            "test_auc": test_auc
        }

    def generate_predictions_csv(self, output_file="xgboost_predictions_new.csv"):
        """Generate predictions CSV with properly formatted timestamps"""
        if self.model is None:
            raise RuntimeError("Model not trained!")

        logger.info("📊 Generating predictions CSV...")
        
        # Get probabilities for test set
        y_test_proba = self.model.predict_proba(self.X_test)
        y_test_pred = self.model.predict(self.X_test)
        
        # Create predictions DataFrame
        predictions_df = pd.DataFrame({
            'timestamp': self.test_timestamps,
            'prob_up': y_test_proba[:, 1],    # Probability of class 1
            'prob_down': y_test_proba[:, 0],  # Probability of class 0
            'winning_prob': np.maximum(y_test_proba[:, 0], y_test_proba[:, 1]),  # Higher probability
            'prediction': y_test_pred,
            'actual': self.y_test
        })
        
        # Ensure timestamp is in datetime format
        predictions_df['timestamp'] = pd.to_datetime(predictions_df['timestamp'])
        
        # Round probabilities to 6 decimal places to match your format
        predictions_df['prob_down'] = predictions_df['prob_down'].round(6)
        predictions_df['prob_up'] = predictions_df['prob_up'].round(6)
        predictions_df['winning_prob'] = predictions_df['winning_prob'].round(6)
        
        # Sort by timestamp to ensure chronological order
        predictions_df = predictions_df.sort_values('timestamp').reset_index(drop=True)
        
        # Save to CSV with proper path
        output_path = os.path.join(OUTPUT_PATH, output_file)
        predictions_df.to_csv(output_path, index=False)
        
        logger.info(f"💾 Predictions saved to: {output_path}")
        logger.info(f"📊 Total predictions: {len(predictions_df)}")
        logger.info(f"📅 Date range: {predictions_df['timestamp'].min()} to {predictions_df['timestamp'].max()}")
        
        # Show sample of predictions
        print("\n🔍 SAMPLE PREDICTIONS:")
        print(predictions_df.head(10).to_string(index=False))
        
        # Calculate accuracy on predictions
        accuracy = accuracy_score(predictions_df['actual'], predictions_df['prediction'])
        precision = precision_score(predictions_df['actual'], predictions_df['prediction'], zero_division=0)
        
        print(f"\n📈 PREDICTION SUMMARY:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Correct predictions: {(predictions_df['prediction'] == predictions_df['actual']).sum()}/{len(predictions_df)}")
        
        # Verify timestamp format
        print(f"\n✅ Timestamp format check:")
        print(f"First timestamp: {predictions_df['timestamp'].iloc[0]}")
        print(f"Last timestamp: {predictions_df['timestamp'].iloc[-1]}")
        print(f"Timestamp dtype: {predictions_df['timestamp'].dtype}")
        
        return predictions_df

    def save_model(self, output_dir="xgboost_model"):
        """Save the trained model"""
        if self.model is None:
            raise RuntimeError("Model not trained!")
        
        model_dir = os.path.join(OUTPUT_PATH, output_dir)
        os.makedirs(model_dir, exist_ok=True)
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save model
        model_path = os.path.join(model_dir, f"xgboost_model_{ts}.joblib")
        joblib.dump(self.model, model_path)
        
        # Save encoders and feature names
        if self.label_encoders:
            joblib.dump(self.label_encoders, os.path.join(model_dir, f"xgboost_encoders_{ts}.joblib"))
        
        joblib.dump(self.feature_columns, os.path.join(model_dir, f"xgboost_features_{ts}.joblib"))
        
        logger.info(f"💾 XGBoost model saved to: {model_path}")
        return model_path


def main():
    """Main function to train XGBoost model and generate predictions"""
    logger.info("🚀 Starting XGBoost training pipeline...")
    
    # Initialize trainer
    trainer = XGBoostTrainer()
    
    try:
        # Load and preprocess data
        trainer.load_and_preprocess()
        
        # Train model
        trainer.train()
        
        # Evaluate model
        metrics = trainer.evaluate()
        
        # Generate predictions CSV
        predictions_df = trainer.generate_predictions_csv("xgboost_predictions_fixed.csv")
        
        # Save model
        model_path = trainer.save_model()
        
        logger.info("🏁 XGBoost training and prediction generation completed!")
        
        # Ask if user wants to replace the original file
        print("\n" + "="*60)
        replace = input("Replace the original xgboost_predictions.csv with the new file? (y/n): ").lower()
        if replace == 'y':
            original_path = os.path.join(OUTPUT_PATH, "xgboost_predictions.csv")
            new_path = os.path.join(OUTPUT_PATH, "xgboost_predictions_fixed.csv")
            
            # Backup original
            backup_path = os.path.join(OUTPUT_PATH, "xgboost_predictions_backup.csv")
            if os.path.exists(original_path):
                import shutil
                shutil.copy2(original_path, backup_path)
                print(f"✓ Original backed up to: xgboost_predictions_backup.csv")
            
            # Replace with new file
            predictions_df.to_csv(original_path, index=False)
            print(f"✓ Original file replaced with fixed version")
        
        return trainer, metrics, predictions_df
        
    except Exception as e:
        logger.error(f"Pipeline failed: {str(e)}")
        raise


if __name__ == "__main__":
    trainer, metrics, predictions_df = main()

INFO:__main__:🚀 Starting XGBoost training pipeline...
INFO:__main__:📥 Loading data for XGBoost training...
INFO:__main__:Loaded data shape: (20718, 67)
INFO:__main__:🕐 Parsing timestamps...
INFO:__main__:Target distribution:
target
1    10751
0     9967
Name: count, dtype: int64
INFO:__main__:Detected binary classification with classes: [0 1]
INFO:__main__:Train set: (16574, 65), Test set: (4144, 65)
INFO:__main__:Features: 65
INFO:__main__:Test date range: 2023-05-07 12:00:00 to 2025-03-28 00:00:00
INFO:__main__:🚀 Training XGBoost model...
INFO:__main__:✓ Training completed!
INFO:__main__:📊 Evaluating XGBoost model...
INFO:__main__:📊 Generating predictions CSV...
INFO:__main__:💾 Predictions saved to: C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Final_runs_csv\xgboost_predictions_fixed.csv
INFO:__main__:📊 Total predictions: 4144
INFO:__main__:📅 Date range: 2023-05-07 12:00:00 to 2025-03-28 00:00:00
INFO:__main__:💾 XGBoost model saved to: C:\Use


🎯 XGBOOST MODEL RESULTS
Training Accuracy:   0.7544
Test Accuracy:       0.5309
Test Precision:      0.5571
Test Recall:         0.4112
Test F1:             0.4732
Test AUC:            0.5434
Overfitting Gap:     0.2235

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.66      0.58      2021
           1       0.56      0.41      0.47      2123

    accuracy                           0.53      4144
   macro avg       0.54      0.53      0.53      4144
weighted avg       0.54      0.53      0.52      4144


🔍 SAMPLE PREDICTIONS:
          timestamp  prob_up  prob_down  winning_prob  prediction  actual
2023-05-07 12:00:00 0.483700   0.516300      0.516300           0       0
2023-05-07 16:00:00 0.444156   0.555844      0.555844           0       0
2023-05-07 20:00:00 0.493198   0.506802      0.506802           0       0
2023-05-08 00:00:00 0.582110   0.417890      0.582110           1       0
2023-05-08 04:00:00 0.650563   0.34

In [2]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Final_runs_csv\xgboost_predictions_fixed.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")


📊 Evaluation at threshold 0.5:
Precision: 0.557
Recall   : 0.411
F1 Score : 0.473
